In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
from haversine import haversine
import requests
import time
import matplotlib.pyplot as plt
from matplotlib import rc 
rc('font', family='AppleGothic')
import base64
import folium
from folium.plugins import MarkerCluster, BeautifyIcon
from folium.features import CustomIcon
import emoji
import numpy as np
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:70% !important; }</style>")) 
from branca.colormap import LinearColormap
import geopandas as gpd

# 0. 데이터 로드

### 0-1. MDM 데이터 로드

In [2]:
the_mdm = pd.read_excel("2020_12_08_거주지정보_MDM.xlsx")
print(the_mdm.shape)
the_mdm.head(2)
the_who_lst = the_mdm.이름.values.tolist()
# len(the_who_lst)

(882, 31)


### 0-2. YTC 데이터 로드

In [3]:
the_ytc = pd.read_excel("2020_12_08_거주지정보_YTC.xlsx")
print(the_ytc.shape)
the_ytc.head(2)
# the_who_lst = the_df.이름.values.tolist()
# len(the_who_lst)

(882, 31)


,이름,대중교통_노선1,대중교통_환승1,대중교통_시간1,대중교통_비용1,대중교통_노선2,대중교통_환승2,대중교통_시간2,대중교통_비용2,대중교통_노선3,...,자차_시간2,자차_거리2주유비용,택시비용,위도,경도,자차_거리2,주유비용,대중교통_평균,자차_평균,거리_평균
0,방시혁,"['6411', '752']","['논현동사거리.학동역', '환승 동작역.국립현충원']",41분,"1,200원","['6411', '151']","['논현동사거리.학동역', '환승 흑석동효사정앞']",41분,"1,200원","['7호선', '501']",...,23분,NaN,"9,800원",37.516488,127.03068,8.78km,909원,43분,22분,8.61km
1,박지원,"['6411', '752']","['단대부고.대치아이파크아파트', '환승 동작역.국립현충원']",49분,"1,300원","['6411', '151']","['단대부고.대치아이파크아파트', '환승 흑석동효사정앞']",49분,"1,300원","['수인분당선', '9호선', '151']",...,31분,NaN,"12,900원",37.494323,127.05351,12.62km,"1,266원",52분,30분,12.19km


### 0-3. MDM & YTC 데이터 Merge

In [4]:
the_df = pd.merge(the_mdm[['이름','위도','경도','대중교통_평균','자차_평균','거리_평균']],
                  the_ytc[['이름','대중교통_평균','자차_평균','거리_평균']],
                  on = "이름", how = "left")
the_df.columns = ['이름', '위도', '경도', 'MDM_대중교통', 'MDM_자차', 'MDM_거리', 'YTC_대중교통',
       'YTC_자차', 'YTC_거리']
the_df

,이름,위도,경도,MDM_대중교통,MDM_자차,MDM_거리,YTC_대중교통,YTC_자차,YTC_거리
0,방시혁,37.516488,127.030680,30분,14분,4.58km,43분,22분,8.61km
1,박지원,37.494323,127.053510,23분,9분,2.55km,52분,30분,12.19km
2,김태호,37.512539,127.036076,29분,12분,4.08km,43분,22분,9.19km
3,이진형,37.464653,127.143821,54분,20분,10.96km,1시간 31분,41분,23.04km
4,김신규,37.514519,127.006996,38분,21분,8.02km,37분,18분,7.33km
...,...,...,...,...,...,...,...,...,...
877,장무현,37.576280,127.222050,1시간 18분,37분,22.4km,1시간 20분,41분,29.54km
878,이수현B,37.375028,127.144698,1시간 13분,39분,20.57km,1시간 8분,45분,26.56km
879,이혜린,37.300538,127.077000,1시간 12분,41분,30.04km,1시간 24분,51분,35.44km
880,권세라B,37.405179,127.120820,57분,28분,15.74km,1시간 25분,39분,25.54km


### 0-4. 출퇴근 데이터 로드

# 1. 데이터 전처리
### 1-1. MDM에서 YTC로 근무지 변경 후, 대중교통, 자차, 거리 diff data 파생

In [5]:
def DERIVE_COLS(the_df):

    the_lst = []
    for sub_lst in the_df[['MDM_대중교통','YTC_대중교통']].values:
        clear_lst = []
        for i in sub_lst:
            if i != '':
                if "분" in i and "시간" not in i:
                    clear_lst.append(int(i.split("분")[0]))
                elif "분" in i and "시간" in i:
                    try:
                        hour = int(re.sub("시외구간 ","",i.split("시간")[0]))
                        minn = int(i.split("시간")[1].strip()[:-1])
                        res = (hour * 60) + minn
                        clear_lst.append(res)
                    except ValueError:
                        res = "알 수 없음"
                        clear_lst.append(res)
                elif "시간" in i and "분" not in i:
                    hour = int(re.sub("시외구간 ","",i.split("시간")[0]))
                    res = hour * 60
                    clear_lst.append(res)
#         the_cal = str(clear_lst[0]-clear_lst[1]) + "분"
        the_cal = clear_lst[1]-clear_lst[0]
        the_lst.append(the_cal)
    the_df['대중교통_차이'] = the_lst
    
    
    the_lst = []
    for sub_lst in the_df[['MDM_자차','YTC_자차']].values:
        clear_lst = []
        for i in sub_lst:
            if i != '':
                if "분" in i and "시간" not in i:
                    clear_lst.append(int(i.split("분")[0]))
                elif "분" in i and "시간" in i:
                    try:
                        hour = int(re.sub("시외구간 ","",i.split("시간")[0]))
                        minn = int(i.split("시간")[1].strip()[:-1])
                        res = (hour * 60) + minn
                        clear_lst.append(res)
                    except ValueError:
                        res = "알 수 없음"
                        clear_lst.append(res)
                elif "시간" in i and "분" not in i:
                    hour = int(re.sub("시외구간 ","",i.split("시간")[0]))
                    res = hour * 60
                    clear_lst.append(res)
#         the_cal = str(clear_lst[0]-clear_lst[1]) + "분"
        the_cal = clear_lst[1]-clear_lst[0]
        the_lst.append(the_cal)
    the_df['자차_차이'] = the_lst
    
    
    the_lst = []
    for sub_lst in the_df[['MDM_거리','YTC_거리']].values:
        clear_lst = []
        for i in sub_lst:
            if "km" in i :
                res = i.split("km")[0]
                clear_lst.append(float(res))
#         the_val = str(round(clear_lst[0] - clear_lst[1],2)) + "km"
        the_val = round(clear_lst[1] - clear_lst[0],2)
        the_lst.append(the_val)
    the_df['거리_차이'] = the_lst
    return the_df

def GET_DIST_COLOR(final_df):
    the_cate_lst = []
    the_color_lst = []

    for who, dist in zip(final_df['이름'], final_df['거리_차이']):

        if dist < -10:
            the_color = "red"
            the_cate = 'worst'
        elif dist >= -10 and dist <-5:
            the_color = "grey"
            the_cate = 'worse'
        elif dist >= -5 and dist < 0:
            the_color = "lightgreen"
            the_cate = 'bad'
        elif dist >= 0 and dist < 5:
            the_color = "chocolate"
            the_cate = "nice"
        elif dist > 5 and dist < 10:
            the_color = "violet"
            the_cate = 'good'
        elif dist >= 10 and dist < 15:
            the_color = "blue"
            the_cate = 'better'
        else:
            the_color = "gold"
            the_cate = 'best'

        the_color_lst.append(the_color)
        the_cate_lst.append(the_cate)

    final_df['컬러구분'] = the_color_lst
    final_df['거리구분'] = the_cate_lst
    return final_df

def GET_COMMUTE(the_final):
    gogo_df = pd.read_excel("1월-11월 구성원_평균_출퇴근기록.xlsx")
    the_final = pd.merge(the_final, gogo_df, on = "이름", how = "left")
    the_final[['출근','퇴근']] = the_final[['출근','퇴근']].fillna("정보없음")
    return the_final

In [6]:
final_df = DERIVE_COLS(the_df)
final_df

,이름,위도,경도,MDM_대중교통,MDM_자차,MDM_거리,YTC_대중교통,YTC_자차,YTC_거리,대중교통_차이,자차_차이,거리_차이
0,방시혁,37.516488,127.030680,30분,14분,4.58km,43분,22분,8.61km,13,8,4.03
1,박지원,37.494323,127.053510,23분,9분,2.55km,52분,30분,12.19km,29,21,9.64
2,김태호,37.512539,127.036076,29분,12분,4.08km,43분,22분,9.19km,14,10,5.11
3,이진형,37.464653,127.143821,54분,20분,10.96km,1시간 31분,41분,23.04km,37,21,12.08
4,김신규,37.514519,127.006996,38분,21분,8.02km,37분,18분,7.33km,-1,-3,-0.69
...,...,...,...,...,...,...,...,...,...,...,...,...
877,장무현,37.576280,127.222050,1시간 18분,37분,22.4km,1시간 20분,41분,29.54km,2,4,7.14
878,이수현B,37.375028,127.144698,1시간 13분,39분,20.57km,1시간 8분,45분,26.56km,-5,6,5.99
879,이혜린,37.300538,127.077000,1시간 12분,41분,30.04km,1시간 24분,51분,35.44km,12,10,5.40
880,권세라B,37.405179,127.120820,57분,28분,15.74km,1시간 25분,39분,25.54km,28,11,9.80


In [7]:
final_df = GET_DIST_COLOR(final_df)
final_df

,이름,위도,경도,MDM_대중교통,MDM_자차,MDM_거리,YTC_대중교통,YTC_자차,YTC_거리,대중교통_차이,자차_차이,거리_차이,컬러구분,거리구분
0,방시혁,37.516488,127.030680,30분,14분,4.58km,43분,22분,8.61km,13,8,4.03,chocolate,nice
1,박지원,37.494323,127.053510,23분,9분,2.55km,52분,30분,12.19km,29,21,9.64,violet,good
2,김태호,37.512539,127.036076,29분,12분,4.08km,43분,22분,9.19km,14,10,5.11,violet,good
3,이진형,37.464653,127.143821,54분,20분,10.96km,1시간 31분,41분,23.04km,37,21,12.08,blue,better
4,김신규,37.514519,127.006996,38분,21분,8.02km,37분,18분,7.33km,-1,-3,-0.69,lightgreen,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,장무현,37.576280,127.222050,1시간 18분,37분,22.4km,1시간 20분,41분,29.54km,2,4,7.14,violet,good
878,이수현B,37.375028,127.144698,1시간 13분,39분,20.57km,1시간 8분,45분,26.56km,-5,6,5.99,violet,good
879,이혜린,37.300538,127.077000,1시간 12분,41분,30.04km,1시간 24분,51분,35.44km,12,10,5.40,violet,good
880,권세라B,37.405179,127.120820,57분,28분,15.74km,1시간 25분,39분,25.54km,28,11,9.80,violet,good


In [8]:
the_final = GET_COMMUTE(final_df)
the_final

,이름,위도,경도,MDM_대중교통,MDM_자차,MDM_거리,YTC_대중교통,YTC_자차,YTC_거리,대중교통_차이,자차_차이,거리_차이,컬러구분,거리구분,출근,퇴근
0,방시혁,37.516488,127.030680,30분,14분,4.58km,43분,22분,8.61km,13,8,4.03,chocolate,nice,정보없음,정보없음
1,박지원,37.494323,127.053510,23분,9분,2.55km,52분,30분,12.19km,29,21,9.64,violet,good,정보없음,정보없음
2,김태호,37.512539,127.036076,29분,12분,4.08km,43분,22분,9.19km,14,10,5.11,violet,good,정보없음,정보없음
3,이진형,37.464653,127.143821,54분,20분,10.96km,1시간 31분,41분,23.04km,37,21,12.08,blue,better,10:30:24,21:45:31
4,김신규,37.514519,127.006996,38분,21분,8.02km,37분,18분,7.33km,-1,-3,-0.69,lightgreen,bad,정보없음,정보없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,장무현,37.576280,127.222050,1시간 18분,37분,22.4km,1시간 20분,41분,29.54km,2,4,7.14,violet,good,정보없음,정보없음
878,이수현B,37.375028,127.144698,1시간 13분,39분,20.57km,1시간 8분,45분,26.56km,-5,6,5.99,violet,good,정보없음,정보없음
879,이혜린,37.300538,127.077000,1시간 12분,41분,30.04km,1시간 24분,51분,35.44km,12,10,5.40,violet,good,10:42:28,23:12:29
880,권세라B,37.405179,127.120820,57분,28분,15.74km,1시간 25분,39분,25.54km,28,11,9.80,violet,good,정보없음,정보없음


In [9]:
from cmath import rect, phase
from math import radians, degrees

def meanAngle(deg):
    complexDegree = sum(rect(1, radians(d)) for d in deg) / len(deg)
    argument = phase(complexDegree)
    meanAngle = degrees(argument)
    return meanAngle

def meanTime(times):
    t = (time.split(':') for time in times)
    seconds = ((float(s) + int(m) * 60 + int(h) * 3600) 
               for h, m, s in t)
    day = 24 * 60 * 60
    toAngles = [s * 360. / day for s in seconds]
    meanAsAngle = meanAngle(toAngles)
    meanSeconds = meanAsAngle * day / 360.
    if meanSeconds < 0:
        meanSeconds += day
    h, m = divmod(meanSeconds, 3600)
    m, s = divmod(m, 60)
    return('%02i:%02i:%02i' % (h, m, s))

print(meanTime(["15:00:00", "21:00:00"]))
# 18:00:00
print(meanTime(["23:00:00", "01:00:00"]))

18:00:00
00:00:00


In [10]:
# who_lst = the_final['이름'].tolist()#[:100]
# GET_THE_MAP2(the_final, who_lst,"거리증감_구분_1214")

In [11]:
# who_lst = the_final['이름'].tolist()#[:100]
# GET_THE_MAP2(the_final, who_lst,"거리증감_구분_1214")

# Insight

In [12]:
df = pd.read_excel("hc20201123_주소근태데이터포함.xlsx")
# df22 = pd.read_excel("hc20201123_주소근태데이터포함.xlsx", sheet_name ='근태수당참고')
# df = pd.merge(df, df22[['이름','1월 오버타임', '2월 오버타임', '3월 오버타임', '4월 오버타임', '5월 오버타임', '6월 오버타임', '7월 오버타임', '8월 오버타임', '9월 오버타임']], on = "이름", how = "left")
# df[['법인','실','팀']] = df[['법인','실','팀']].fillna("-")

# print(df.shape)
# df.head()

In [13]:
the_final = pd.merge(df, the_final[['이름','위도','경도','MDM_대중교통', 'MDM_자차', 'MDM_거리', 'YTC_대중교통', 'YTC_자차',
       'YTC_거리', '대중교통_차이', '자차_차이', '거리_차이','출근','퇴근']])
print(the_final.shape)
the_final.head(3)

(882, 52)


,New 사번,재직구분,이름,이메일,파싱용 주소,주소,1-9월\n평균근무시간 (만근월 기준),총OT시간,평균OT시간\n(총OT/OT한 개월수),1-9 OT지급총액,...,MDM_자차,MDM_거리,YTC_대중교통,YTC_자차,YTC_거리,대중교통_차이,자차_차이,거리_차이,출근,퇴근
0,BHE0502001,재직,방시혁,hitman@bighitcorp.com,서울 강남구 논현동 80-12 404,서울 강남구 논현동 80-12 404,NaN,NaN,NaN,NaN,...,14분,4.58km,43분,22분,8.61km,13,8,4.03,정보없음,정보없음
1,BHE2005004,재직,박지원,jwpark@bighitcorp.com,서울 강남구 선릉로 217 (도곡동) 411동 1004호,서울 강남구 선릉로 217 (도곡동) 411동 1004호,NaN,NaN,NaN,NaN,...,9분,2.55km,52분,30분,12.19km,29,21,9.64,정보없음,정보없음
2,BHE1903010,재직,김태호,ian@bighitcorp.com,서울 강남구 언주로 626 로얄팰리스 1201호,서울 강남구 언주로 626 로얄팰리스 1201호,NaN,NaN,NaN,NaN,...,12분,4.08km,43분,22분,9.19km,14,10,5.11,정보없음,정보없음


In [14]:
# df.to_excel("**출퇴근소요시간_20201214.xlsx", index = False, encoding = "utf-8-sig")

In [15]:
def GET_AREA(df):
    the_val_lst= []
    for i in df.주소:
        if "경기도부천시심중로" in i:
            i = "경기도 부천시 심중로"
        city = i.split(" ")[0]
        try:
            area = i.split(" ")[1]
        except IndexError:
            area = i
            print("************", area)
    #     the_val = df_geo['area'][df_geo['area'].str.contains(area)].values[0]
    #     except IndexError:
    #         area = i.split(" ")[1].split("대로")[0]
    #         the_val = df_geo['area'][df_geo['area'].str.contains(area)].values[0]
        if "서울" in city:
            city = "서울"
            area = i.split(" ")[1]
            the_val = city + " "+area
        elif '경기' in city:
            city = i.split(" ")[1]
            area = i.split(" ")[2]
            the_val = city
        elif "인천" in city:
            city = "인천"
            area = i.split(" ")[1]
            the_val = city + " "+area
        else:
            if "도" in city:
                city = i.split(" ")[1]
    #             area = i.split(" ")[1]
                the_val = city #+ " "+area
                print("$$$$$$", the_val)
            elif "시" in city:
                the_val = city
    #             print("***********************", city)
        if "디지털로" in the_val:
            the_val = "광명시"
        elif "홍은2동" in the_val:
            the_val = "서울시 서대문구"
        elif "오남읍" in the_val:
            the_val = "남양주시"
        elif "송파대로" in the_val:
            the_val = "서울 송파구"
        elif "서초중앙로" in the_val:
            the_val = "서울 서초구"
        elif "서울시 서대문" in the_val:
            the_val = "서울 서대문구"
        elif "서울 삼성동" in the_val:
            the_val = "서울 강남구"
        elif "서울 신림동" in the_val:
            the_val = "서울 관악구"
        elif "서울 잠실동" in the_val:
            the_val = "서울 송파구"
        elif "성남" in the_val:
            val = i.split("성남시")[1].split(" ")[1]#the_val.split("성남시")[1].split(" ")[0]#.strip()
            if '운중동' in i:
                the_val = "성남 분당구"
            else:
                the_val = "성남 "+val
            print("!!!",the_val)

    #     print("!!!",city, area, "--->", the_val)

        the_val_lst.append(the_val)
    #     if "서울" not in 
    df['지역'] = the_val_lst
    return df

In [16]:
# df['지역'] = the_val_lst
# df.head()

In [17]:
the_final = GET_AREA(the_final)
the_final

!!! 성남 수정구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 수정구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 수정구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 수정구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 수정구
!!! 성남 수정구
!!! 성남 분당구
!!! 성남 중원구
!!! 성남 수정구
!!! 성남 중원구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 수정구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 중원구
!!! 성남 분당구
!!! 성남 분당구
!!! 성남 수정구


,New 사번,재직구분,이름,이메일,파싱용 주소,주소,1-9월\n평균근무시간 (만근월 기준),총OT시간,평균OT시간\n(총OT/OT한 개월수),1-9 OT지급총액,...,MDM_거리,YTC_대중교통,YTC_자차,YTC_거리,대중교통_차이,자차_차이,거리_차이,출근,퇴근,지역
0,BHE0502001,재직,방시혁,hitman@bighitcorp.com,서울 강남구 논현동 80-12 404,서울 강남구 논현동 80-12 404,NaN,NaN,NaN,NaN,...,4.58km,43분,22분,8.61km,13,8,4.03,정보없음,정보없음,서울 강남구
1,BHE2005004,재직,박지원,jwpark@bighitcorp.com,서울 강남구 선릉로 217 (도곡동) 411동 1004호,서울 강남구 선릉로 217 (도곡동) 411동 1004호,NaN,NaN,NaN,NaN,...,2.55km,52분,30분,12.19km,29,21,9.64,정보없음,정보없음,서울 강남구
2,BHE1903010,재직,김태호,ian@bighitcorp.com,서울 강남구 언주로 626 로얄팰리스 1201호,서울 강남구 언주로 626 로얄팰리스 1201호,NaN,NaN,NaN,NaN,...,4.08km,43분,22분,9.19km,14,10,5.11,정보없음,정보없음,서울 강남구
3,BHE1801001,재직,이진형,aj@bighitcorp.com,경기도 성남시 수정구 위례동로 15,경기도 성남시 수정구 위례동로 15,NaN,NaN,NaN,NaN,...,10.96km,1시간 31분,41분,23.04km,37,21,12.08,10:30:24,21:45:31,성남 수정구
4,BHE0807001,재직,김신규,bighit4832@bighitcorp.com,서울시 서초구 잠원로117 \n아크로리버뷰 104동2204호,서울시 서초구 잠원로117 \n아크로리버뷰 104동2204호,NaN,NaN,NaN,NaN,...,8.02km,37분,18분,7.33km,-1,-3,-0.69,정보없음,정보없음,서울 서초구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,PLD1911005,재직,박지혜B,jhpark@pledis.co.kr,서울시 종로구 소격동 35-2,서울시 종로구 소격동 35-2,NaN,NaN,NaN,NaN,...,14.25km,37분,23분,8.8km,-24,-13,-5.45,정보없음,정보없음,서울 종로구
878,PLD1808002,재직,가하,kahastar@pledis.co.kr,서울시 양천구 신정동 944-4,서울시 양천구 신정동 944-4,NaN,NaN,NaN,NaN,...,26.77km,46분,21분,13.57km,-24,-23,-13.20,10:40:41,21:12:47,서울 양천구
879,PLD1707001,재직,강다은B,kangda93@pledis.co.kr,경기도 의왕시 포일동 이미로 17 포일숲속마을,경기도 의왕시 포일동 이미로 17. 포일숲속마을 2단지 210동 603호,NaN,NaN,NaN,NaN,...,22.88km,1시간 3분,37분,21.18km,-6,0,-1.70,정보없음,정보없음,의왕시
880,PLD1801001,재직,정다예,dyjeong@pledis.co.kr,서울시 동작구 대방동411-33 팰리스타운 602호,서울시 동작구 대방동411-33 팰리스타운 602호,NaN,NaN,NaN,NaN,...,19.86km,29분,14분,6.11km,-26,-21,-13.75,10:20:44,21:31:01,서울 동작구


In [18]:
# df22 = df[['이름','법인','실','팀','파트','직책','겸직 여부','그룹입사일','위도','경도','성별',
#            '1-9월\n평균근무시간 (만근월 기준)','총OT시간','평균OT시간\n(총OT/OT한 개월수)','1-9 OT지급총액','1-9 OT수당평균\n(총지급액/지급개월수)',
#            'REV 직군','REV 직무/역할','REV 직렬','REV Position','1월 오버타임', '2월 오버타임', '3월 오버타임', '4월 오버타임', '5월 오버타임', '6월 오버타임', '7월 오버타임', '8월 오버타임', '9월 오버타임',
#            'MDM_대중교통', 'MDM_자차', 'MDM_거리', 'YTC_대중교통', 'YTC_자차','YTC_거리', '대중교통_차이', '자차_차이', '거리_차이','출근','퇴근','지역']]
# print(df22.shape)
# df22.head()

In [19]:
def GET_THAT(df):
    pub_lst = []
    for i in df['YTC_대중교통']:
        if '시간' not in i and '분' in i:
            the_val = int(i.split("분")[0].strip())
        elif '시간' in i and '분' in i:
            the_hour = int(i.split("시간")[0]) * 60
            the_minn = int(i.split("시간")[1].split("분")[0].strip())
            the_val = the_hour + the_minn
        elif '시간'in i and '분' not in i:
            the_val = int(i.split('시간')[0].strip()) * 60
        else:
            print("!!!!!!!!!!!", i)
        pub_lst.append(the_val)

    pri_lst = []
    for i in df['YTC_자차']:
        if '시간' not in i and '분' in i:
            the_val = int(i.split("분")[0].strip())
        elif '시간' in i and '분' in i:
            the_hour = int(i.split("시간")[0]) * 60
            the_minn = int(i.split("시간")[1].split("분")[0].strip())
            the_val = the_hour + the_minn
        elif '시간'in i and '분' not in i:
            the_val = int(i.split('시간')[0].strip()) * 60
        else:
            print("!!!!!!!!!!!", i)
        pri_lst.append(the_val)


    pub_lst2 = []
    for i in df['MDM_대중교통']:
        if '시간' not in i and '분' in i:
            the_val = int(i.split("분")[0].strip())
        elif '시간' in i and '분' in i:
            the_hour = int(i.split("시간")[0]) * 60
            the_minn = int(i.split("시간")[1].split("분")[0].strip())
            the_val = the_hour + the_minn
        elif '시간'in i and '분' not in i:
            the_val = int(i.split('시간')[0].strip()) * 60
        else:
            print("!!!!!!!!!!!", i)
        pub_lst2.append(the_val)

    pri_lst2 = []
    for i in df['MDM_자차']:
        if '시간' not in i and '분' in i:
            the_val = int(i.split("분")[0].strip())
        elif '시간' in i and '분' in i:
            the_hour = int(i.split("시간")[0]) * 60
            the_minn = int(i.split("시간")[1].split("분")[0].strip())
            the_val = the_hour + the_minn
        elif '시간'in i and '분' not in i:
            the_val = int(i.split('시간')[0].strip()) * 60
        else:
            print("!!!!!!!!!!!", i)
        pri_lst2.append(the_val)


    df['MDM_거리2'] = [float(i.split("km")[0]) for i in df['MDM_거리']]
    df['YTC_거리2'] = [float(i.split("km")[0]) for i in df['YTC_거리']]
    df['YTC_대중교통2'] = pub_lst
    df['YTC_자차2'] = pri_lst
    df['MDM_대중교통2'] = pub_lst2
    df['MDM_자차2'] = pri_lst2
    # the_df['age'] = the_df.age.astype(float)
    return df

In [20]:
the_final = GET_THAT(the_final)
the_final

,New 사번,재직구분,이름,이메일,파싱용 주소,주소,1-9월\n평균근무시간 (만근월 기준),총OT시간,평균OT시간\n(총OT/OT한 개월수),1-9 OT지급총액,...,거리_차이,출근,퇴근,지역,MDM_거리2,YTC_거리2,YTC_대중교통2,YTC_자차2,MDM_대중교통2,MDM_자차2
0,BHE0502001,재직,방시혁,hitman@bighitcorp.com,서울 강남구 논현동 80-12 404,서울 강남구 논현동 80-12 404,NaN,NaN,NaN,NaN,...,4.03,정보없음,정보없음,서울 강남구,4.58,8.61,43,22,30,14
1,BHE2005004,재직,박지원,jwpark@bighitcorp.com,서울 강남구 선릉로 217 (도곡동) 411동 1004호,서울 강남구 선릉로 217 (도곡동) 411동 1004호,NaN,NaN,NaN,NaN,...,9.64,정보없음,정보없음,서울 강남구,2.55,12.19,52,30,23,9
2,BHE1903010,재직,김태호,ian@bighitcorp.com,서울 강남구 언주로 626 로얄팰리스 1201호,서울 강남구 언주로 626 로얄팰리스 1201호,NaN,NaN,NaN,NaN,...,5.11,정보없음,정보없음,서울 강남구,4.08,9.19,43,22,29,12
3,BHE1801001,재직,이진형,aj@bighitcorp.com,경기도 성남시 수정구 위례동로 15,경기도 성남시 수정구 위례동로 15,NaN,NaN,NaN,NaN,...,12.08,10:30:24,21:45:31,성남 수정구,10.96,23.04,91,41,54,20
4,BHE0807001,재직,김신규,bighit4832@bighitcorp.com,서울시 서초구 잠원로117 \n아크로리버뷰 104동2204호,서울시 서초구 잠원로117 \n아크로리버뷰 104동2204호,NaN,NaN,NaN,NaN,...,-0.69,정보없음,정보없음,서울 서초구,8.02,7.33,37,18,38,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,PLD1911005,재직,박지혜B,jhpark@pledis.co.kr,서울시 종로구 소격동 35-2,서울시 종로구 소격동 35-2,NaN,NaN,NaN,NaN,...,-5.45,정보없음,정보없음,서울 종로구,14.25,8.80,37,23,61,36
878,PLD1808002,재직,가하,kahastar@pledis.co.kr,서울시 양천구 신정동 944-4,서울시 양천구 신정동 944-4,NaN,NaN,NaN,NaN,...,-13.20,10:40:41,21:12:47,서울 양천구,26.77,13.57,46,21,70,44
879,PLD1707001,재직,강다은B,kangda93@pledis.co.kr,경기도 의왕시 포일동 이미로 17 포일숲속마을,경기도 의왕시 포일동 이미로 17. 포일숲속마을 2단지 210동 603호,NaN,NaN,NaN,NaN,...,-1.70,정보없음,정보없음,의왕시,22.88,21.18,63,37,69,37
880,PLD1801001,재직,정다예,dyjeong@pledis.co.kr,서울시 동작구 대방동411-33 팰리스타운 602호,서울시 동작구 대방동411-33 팰리스타운 602호,NaN,NaN,NaN,NaN,...,-13.75,10:20:44,21:31:01,서울 동작구,19.86,6.11,29,14,55,35


In [21]:
city_lst = []
for city in the_final['지역']:
    if "인천" in city:
        city = "인천"
        city_lst.append(city)
    else:
        city_lst.append(city)
        
the_final['지역'] = city_lst
the_final.head()

,New 사번,재직구분,이름,이메일,파싱용 주소,주소,1-9월\n평균근무시간 (만근월 기준),총OT시간,평균OT시간\n(총OT/OT한 개월수),1-9 OT지급총액,...,거리_차이,출근,퇴근,지역,MDM_거리2,YTC_거리2,YTC_대중교통2,YTC_자차2,MDM_대중교통2,MDM_자차2
0,BHE0502001,재직,방시혁,hitman@bighitcorp.com,서울 강남구 논현동 80-12 404,서울 강남구 논현동 80-12 404,NaN,NaN,NaN,NaN,...,4.03,정보없음,정보없음,서울 강남구,4.58,8.61,43,22,30,14
1,BHE2005004,재직,박지원,jwpark@bighitcorp.com,서울 강남구 선릉로 217 (도곡동) 411동 1004호,서울 강남구 선릉로 217 (도곡동) 411동 1004호,NaN,NaN,NaN,NaN,...,9.64,정보없음,정보없음,서울 강남구,2.55,12.19,52,30,23,9
2,BHE1903010,재직,김태호,ian@bighitcorp.com,서울 강남구 언주로 626 로얄팰리스 1201호,서울 강남구 언주로 626 로얄팰리스 1201호,NaN,NaN,NaN,NaN,...,5.11,정보없음,정보없음,서울 강남구,4.08,9.19,43,22,29,12
3,BHE1801001,재직,이진형,aj@bighitcorp.com,경기도 성남시 수정구 위례동로 15,경기도 성남시 수정구 위례동로 15,NaN,NaN,NaN,NaN,...,12.08,10:30:24,21:45:31,성남 수정구,10.96,23.04,91,41,54,20
4,BHE0807001,재직,김신규,bighit4832@bighitcorp.com,서울시 서초구 잠원로117 \n아크로리버뷰 104동2204호,서울시 서초구 잠원로117 \n아크로리버뷰 104동2204호,NaN,NaN,NaN,NaN,...,-0.69,정보없음,정보없음,서울 서초구,8.02,7.33,37,18,38,21


In [22]:
# the_final = the_final[the_final['법인']!= 'BNX']

In [23]:
aaa = round(the_final[['MDM_거리2','MDM_자차2','MDM_대중교통2','YTC_거리2','YTC_자차2','YTC_대중교통2','대중교통_차이','자차_차이','거리_차이','지역']].groupby("지역").mean().reset_index(),2)#.to_excel("ex2.xlsx",index = False, encoding = "utf-8-sig")
aaa

,지역,MDM_거리2,MDM_자차2,MDM_대중교통2,YTC_거리2,YTC_자차2,YTC_대중교통2,대중교통_차이,자차_차이,거리_차이
0,고양시,40.57,53.62,101.93,27.67,41.55,75.86,-26.07,-12.07,-12.89
1,과천시,14.14,30.00,57.67,15.97,28.33,51.67,-6.00,-1.67,1.83
2,광명시,27.07,47.40,69.00,15.84,36.00,54.60,-14.40,-11.40,-11.23
3,광주시,28.30,37.17,84.67,40.72,55.00,102.00,17.33,17.83,12.42
4,구리시,18.45,26.80,66.80,24.89,35.40,83.20,16.40,8.60,6.44
5,군포시,26.94,41.25,73.75,28.78,43.75,67.50,-6.25,2.50,1.83
6,김포시,44.06,53.30,114.00,32.03,40.90,87.50,-26.50,-12.40,-12.03
7,남양주시,26.29,36.47,81.16,33.61,45.32,94.05,12.89,8.84,7.32
8,대전광역시,155.56,137.00,117.00,167.66,166.00,117.00,0.00,29.00,12.10
9,부산광역시,404.94,272.00,245.00,414.51,310.00,245.00,0.00,38.00,9.57


In [24]:
for city in aaa.지역:
    print(the_final['출근'][the_final['지역'] == city].values)

['10:25:10' '09:53:17' '10:50:44' '09:45:11' '10:39:21' '09:45:56'
 '11:45:57' '12:30:24' '정보없음' '08:43:18' '09:57:45' '09:18:53' '10:32:31'
 '정보없음' '11:26:36' '09:55:25' '10:01:21' '09:39:13' '11:03:55' '10:52:56'
 '09:04:15' '09:48:16' '08:35:05' '정보없음' '10:57:09' '09:31:10' '10:00:31'
 '정보없음' '정보없음']
['10:44:43' '11:02:06' '12:02:28']
['09:52:49' '11:02:42' '10:36:03' '09:43:03' '09:32:08']
['09:59:35' '09:38:24' '09:47:50' '11:05:45' '09:59:19' '09:05:36']
['11:06:34' '09:02:34' '12:05:37' '정보없음' '정보없음']
['08:49:58' '11:12:20' '10:17:41' '08:04:31']
['10:22:11' '09:50:10' '정보없음' '정보없음' '11:08:59' '09:57:02' '09:56:43'
 '08:50:15' '10:34:49' '09:40:49']
['13:35:03' '10:15:17' '정보없음' '09:50:09' '10:28:19' '12:12:18' '정보없음'
 '10:55:33' '10:28:49' '09:39:55' '10:21:35' '10:16:51' '정보없음' '10:46:56'
 '정보없음' '09:45:55' '10:05:07' '07:59:44' '정보없음']
['09:15:31']
['08:20:50']
['10:29:59' '10:52:42' '10:23:55' '11:41:53' '08:11:30' '09:24:53'
 '09:56:16' '09:48:06' '11:59:09' '정보없음' '09:34:3

In [25]:
bbb = the_final[['이름','지역']].groupby("지역").count().reset_index()
bbb = bbb.rename({"이름":"인원수"}, axis = 1)
bbb

,지역,인원수
0,고양시,29
1,과천시,3
2,광명시,5
3,광주시,6
4,구리시,5
5,군포시,4
6,김포시,10
7,남양주시,19
8,대전광역시,1
9,부산광역시,1


In [26]:
aaa = pd.merge(aaa, bbb, on = "지역", how = "left")
aaa

,지역,MDM_거리2,MDM_자차2,MDM_대중교통2,YTC_거리2,YTC_자차2,YTC_대중교통2,대중교통_차이,자차_차이,거리_차이,인원수
0,고양시,40.57,53.62,101.93,27.67,41.55,75.86,-26.07,-12.07,-12.89,29
1,과천시,14.14,30.00,57.67,15.97,28.33,51.67,-6.00,-1.67,1.83,3
2,광명시,27.07,47.40,69.00,15.84,36.00,54.60,-14.40,-11.40,-11.23,5
3,광주시,28.30,37.17,84.67,40.72,55.00,102.00,17.33,17.83,12.42,6
4,구리시,18.45,26.80,66.80,24.89,35.40,83.20,16.40,8.60,6.44,5
5,군포시,26.94,41.25,73.75,28.78,43.75,67.50,-6.25,2.50,1.83,4
6,김포시,44.06,53.30,114.00,32.03,40.90,87.50,-26.50,-12.40,-12.03,10
7,남양주시,26.29,36.47,81.16,33.61,45.32,94.05,12.89,8.84,7.32,19
8,대전광역시,155.56,137.00,117.00,167.66,166.00,117.00,0.00,29.00,12.10,1
9,부산광역시,404.94,272.00,245.00,414.51,310.00,245.00,0.00,38.00,9.57,1


In [27]:
ent_lst = []
out_lst = []
for city in aaa.지역:
    try:
        the_ent = meanTime([i for i in the_final['출근'][the_final['지역'] == city].values if i !='정보없음'])
    except ZeroDivisionError:
        the_ent = '정보없음'
    try:
        the_out = meanTime([i for i in the_final['퇴근'][the_final['지역'] == city].values if i != '정보없음'])
    except ZeroDivisionError:
        the_out = '정보없음'
    ent_lst.append(the_ent)
    out_lst.append(the_out)
aaa['평균출근'] = ent_lst
aaa['평균퇴근'] = out_lst
aaa

,지역,MDM_거리2,MDM_자차2,MDM_대중교통2,YTC_거리2,YTC_자차2,YTC_대중교통2,대중교통_차이,자차_차이,거리_차이,인원수,평균출근,평균퇴근
0,고양시,40.57,53.62,101.93,27.67,41.55,75.86,-26.07,-12.07,-12.89,29,10:12:25,20:44:35
1,과천시,14.14,30.00,57.67,15.97,28.33,51.67,-6.00,-1.67,1.83,3,11:16:21,21:26:57
2,광명시,27.07,47.40,69.00,15.84,36.00,54.60,-14.40,-11.40,-11.23,5,10:09:17,20:16:57
3,광주시,28.30,37.17,84.67,40.72,55.00,102.00,17.33,17.83,12.42,6,09:55:58,20:09:37
4,구리시,18.45,26.80,66.80,24.89,35.40,83.20,16.40,8.60,6.44,5,10:45:30,22:38:02
5,군포시,26.94,41.25,73.75,28.78,43.75,67.50,-6.25,2.50,1.83,4,09:36:02,20:44:14
6,김포시,44.06,53.30,114.00,32.03,40.90,87.50,-26.50,-12.40,-12.03,10,10:02:38,20:13:42
7,남양주시,26.29,36.47,81.16,33.61,45.32,94.05,12.89,8.84,7.32,19,10:27:45,21:10:32
8,대전광역시,155.56,137.00,117.00,167.66,166.00,117.00,0.00,29.00,12.10,1,09:15:31,21:08:04
9,부산광역시,404.94,272.00,245.00,414.51,310.00,245.00,0.00,38.00,9.57,1,08:20:50,23:55:10


In [28]:
def GET_LATLON(addr_from):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr_from

    headers = {"Authorization":"KakaoAK"+" c2cc737d485c6fa0d6f3cfaa7f44068d"}
    r = requests.get(url, headers=headers)
    the_addr = r.json()["documents"][0]["address"]
    
    return float(the_addr["x"]), float(the_addr["y"])

In [29]:
def GET_KAKAO_GEO(addr_from):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + addr_from
    headers = {"Authorization":"KakaoAK"+" c2cc737d485c6fa0d6f3cfaa7f44068d"}
    r = requests.get(url, headers=headers)
#     return r.json()
    the_addr = r.json()["documents"][0]#["address"]
    return float(the_addr["x"]), float(the_addr["y"])


In [30]:
GET_KAKAO_GEO("삼성역")

(127.063025473533, 37.5088227390184)

In [31]:
def GET_POLY_AREA(city, country):
    url = 'https://nominatim.openstreetmap.org/search.php?q='+city+'+'+country+'&polygon_geojson=1&format=json'
    r = requests.get(url)
    return r.json()[0]['geojson']['coordinates']

In [32]:
the_dict = {}

for i in aaa.지역.unique().tolist():
    the_dict[i] = GET_LATLON(i)


the_lst = []
for i in aaa.지역:
    the_lst.append(the_dict[i])
aaa['위경도'] = the_lst
aaa

,지역,MDM_거리2,MDM_자차2,MDM_대중교통2,YTC_거리2,YTC_자차2,YTC_대중교통2,대중교통_차이,자차_차이,거리_차이,인원수,평균출근,평균퇴근,위경도
0,고양시,40.57,53.62,101.93,27.67,41.55,75.86,-26.07,-12.07,-12.89,29,10:12:25,20:44:35,"(126.831955109648, 37.6584180954929)"
1,과천시,14.14,30.00,57.67,15.97,28.33,51.67,-6.00,-1.67,1.83,3,11:16:21,21:26:57,"(126.987711960677, 37.4292291986772)"
2,광명시,27.07,47.40,69.00,15.84,36.00,54.60,-14.40,-11.40,-11.23,5,10:09:17,20:16:57,"(126.864647815161, 37.4785674780314)"
3,광주시,28.30,37.17,84.67,40.72,55.00,102.00,17.33,17.83,12.42,6,09:55:58,20:09:37,"(127.255138719721, 37.4293941934866)"
4,구리시,18.45,26.80,66.80,24.89,35.40,83.20,16.40,8.60,6.44,5,10:45:30,22:38:02,"(127.129646382723, 37.5943595777621)"
5,군포시,26.94,41.25,73.75,28.78,43.75,67.50,-6.25,2.50,1.83,4,09:36:02,20:44:14,"(126.935205932727, 37.3616318289596)"
6,김포시,44.06,53.30,114.00,32.03,40.90,87.50,-26.50,-12.40,-12.03,10,10:02:38,20:13:42,"(126.715657922927, 37.6152686898333)"
7,남양주시,26.29,36.47,81.16,33.61,45.32,94.05,12.89,8.84,7.32,19,10:27:45,21:10:32,"(127.21648030719, 37.6360272346499)"
8,대전광역시,155.56,137.00,117.00,167.66,166.00,117.00,0.00,29.00,12.10,1,09:15:31,21:08:04,"(127.38483484675, 36.3505388992836)"
9,부산광역시,404.94,272.00,245.00,414.51,310.00,245.00,0.00,38.00,9.57,1,08:20:50,23:55:10,"(129.075087492149, 35.1798200522868)"


In [33]:
# who_lst = [i for i in aaa.지역]
# GET_THE_MAP2(aaa, who_lst, "filename")

In [34]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

def GET_THE_MAP2(the_df, who_lst, filename):
    
    YTC = ['37.5245456464592', '126.964197674537']
    MDM = ['37.5068182254549', '127.066242271781']
#     {'darkred', 'beige', 'darkpurple', 'darkblue', 'cadetblue', 'darkgreen', 'blue', 'black', 'lightgreen', 'lightblue', 'white', 'gray', 'lightgray', 'red', 'green', 'pink', 'orange', 'lightred', 'purple'}.
    the_map = folium.Map(location=YTC, tiles='CartoDB dark_matter', zoom_start=10)

    folium.Marker(location=YTC,  tooltip = "용산트레이드센터", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)
    
    folium.Marker(location=MDM,  tooltip = "엠디엠타워", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)
    
    for km in [10000,20000,30000,40000,50000]:

        folium.Circle(location=YTC,
                      tooltip = str(km)[:2]+"km (직선거리)",
                      radius= km, 
                      color = "red", 
                      weight = 1
                     ).add_to(the_map)

    feature_group = folium.FeatureGroup("Locations")
#     the_loc_lst = []
    for who in who_lst:
        the_who = who
        the_cnt = str(the_df['인원수'][the_df['지역'] == who].values[0])
#         the_who = emoji.emojize(":bust_in_silhouette: "+ who)
#         pub_avg = emoji.emojize("<br>:railway_car: "+ " MDM "+the_df['MDM_대중교통'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_대중교통'][the_df.이름 == str(who)].values[0])
#         self_avg =  emoji.emojize("<br>:automobile: "+ " MDM "+the_df['MDM_자차'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_자차'][the_df.이름 == str(who)].values[0])
#         dist_avg = emoji.emojize("<br>:hourglass_not_done:"+" MDM "+the_df['MDM_거리'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_거리'][the_df.이름 == str(who)].values[0])
#         the_in = emoji.emojize("<br>:sunrise: "+ the_df['출근'][the_df.이름 == str(who)].values[0])
#         the_out = emoji.emojize("<br>:bridge_at_night: "+ the_df['퇴근'][the_df.이름 == str(who)].values[0])
        the_label = the_who+"\n"+the_cnt#str(the_who) + pub_avg + self_avg + dist_avg + the_in + the_out

        the_loc_lst = list(the_df[['위경도']][the_df['지역'] == str(who)].values[0].tolist()[0])
        the_loc = [the_loc_lst[1], the_loc_lst[0]]
        iframe = folium.IFrame(the_label, width=300, height=200)
        popup = folium.Popup(iframe, max_width=300)
        
        folium.CircleMarker(the_loc, radius = int(the_cnt)*0.3, color='#3186cc', fill_color='#3186cc', popup = popup).add_to(the_map)
        
#         the_loc_lst.append(the_loc)
    
    # red, grey, lightgreen, brown, pink(violet), blue, gold,
#     the_map= add_categorical_legend(the_map, '거리 가감구분 (구성원수)',
#                                     colors = ['#FF0000','#A9A9A9','#90EE90',
#                                               '#D2691E','#EE82EE','#0000FF','#FFD700'],
#                                     labels = ['10km 이상 증가(122명)', '5~10km 증가(215명)', '0~5km 증가(133명)',
#                                               '0~5km 감소(52명)', '5~10km 감소(75명)','10~15km 감소(271명)','15km 이상 감소 (14명)'])
    
#     the_map.save("*MAP_"+filename+'.html')
    return the_map
    

In [35]:
import json
from shapely.geometry import Polygon, Point
with open("TL_SCCO_SIG.json") as json_file:
    the_dict = json.load(json_file)
df_geo = pd.DataFrame()
area_lst = []
code_lst = []
poly_lst = []
for i in the_dict['features']:
    the_loca = i['properties']['SIG_KOR_NM']
    the_code = i['properties']['SIG_CD']
    the_geo = i['geometry']['coordinates'][0]
    the_poly = Polygon(the_geo)
    area_lst.append(the_loca)
    code_lst.append(the_code)
    poly_lst.append(the_geo)
    
    
df_geo['area'] = area_lst
df_geo['code'] = code_lst
df_geo['poly'] = poly_lst
df_geo.head()
#     the_point = Point(126.6032113, 37.64197486)
#     if the_poly.contains(the_point) == True:
#         print(the_loca, the_code)


,area,code,poly
0,춘천시,42110,"[[127.60472638427933, 38.072648233869785], [12..."
1,원주시,42130,"[[128.10856018737485, 37.29880426255525], [128..."
2,강릉시,42150,"[[129.04785455302166, 37.64960752653576], [129..."
3,동해시,42170,"[[129.15959017011357, 37.47574351318527], [129..."
4,태백시,42190,"[[129.01210390848698, 37.310116310041145], [12..."


In [36]:
area_lst = aaa.지역.unique().tolist()
print(area_lst)

['고양시', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시', '대전광역시', '부산광역시', '부천시', '서울 강남구', '서울 강동구', '서울 강북구', '서울 강서구', '서울 관악구', '서울 광진구', '서울 구로구', '서울 금천구', '서울 노원구', '서울 도봉구', '서울 동대문구', '서울 동작구', '서울 마포구', '서울 서대문구', '서울 서초구', '서울 성동구', '서울 성북구', '서울 송파구', '서울 양천구', '서울 영등포구', '서울 용산구', '서울 은평구', '서울 종로구', '서울 중구', '서울 중랑구', '서울시 서대문구', '성남 분당구', '성남 수정구', '성남 중원구', '수원시', '시흥시', '안산시', '안성시', '안양시', '양평군', '여주시', '용인시', '울산광역시', '의왕시', '의정부시', '이천시', '인천', '파주시', '포천시', '하남시', '화성시']


In [37]:
df_geo[df_geo['area'].str.contains('성남')]

,area,code,poly
22,성남시 수정구,41131,"[[127.14352658768819, 37.47393041843933], [127..."
23,성남시 중원구,41133,"[[127.18199646591572, 37.467112093650286], [12..."
24,성남시 분당구,41135,"[[127.11920592764365, 37.41566096342704], [127..."


In [38]:
ok_lst = []
poly_lst = []
for area in area_lst:
    if area in [i for i in df_geo.area]:
        ok_lst.append(area)
        poly_lst.append(df_geo['poly'][df_geo.area.str.contains(area)].tolist())
    else:
        if "서울" in area:
            if "강서구" in area:
                poly_lst.append([df_geo['poly'][df_geo['area'].str.contains('강서구')].iloc[1]])
            elif "중구" in area:
                poly_lst.append([df_geo['poly'][df_geo['area'].str.contains('중구')].iloc[3]])
            else:
                gu = area.split(" ")[1]
                ok_lst.append(area)
                poly_lst.append(df_geo['poly'][df_geo.area.str.contains(gu)].tolist())
        elif "인천" in area:
            ok_lst.append("인천")
            rrr = GET_POLY_AREA("인천", 'korea')
            poly_lst.append(rrr[1])
        elif "광역시" in area:
            poly_lst.append("none")
        else:
            print(area)
            if '성남' in area:
                area = area.split(" ")[1]
                rrr = GET_POLY_AREA(area, 'korea')
                ok_lst.append("성남")
                poly_lst.append(rrr)
            else:
                rrr = GET_POLY_AREA("경기도 "+area, 'korea')
                ok_lst.append(area)
#             
                if '안산' in area:
                    poly_lst.append(rrr[1])
                else:
                    poly_lst.append(rrr)


고양시
성남 분당구
성남 수정구
성남 중원구
수원시
안산시
안양시
용인시


In [39]:
def GET_POLY_AREA(city, country):

    url = 'https://nominatim.openstreetmap.org/search.php?q='+city+'+'+country+'&polygon_geojson=1&format=json'
    r = requests.get(url)
    return r.json()[0]['geojson']['coordinates']

In [40]:
# rrr[1]

In [41]:
# rrr = GET_POLY_AREA("경기도 성남시", 'korea')
# rrr

In [42]:
aaa['poly'] = poly_lst
aaa.head()

,지역,MDM_거리2,MDM_자차2,MDM_대중교통2,YTC_거리2,YTC_자차2,YTC_대중교통2,대중교통_차이,자차_차이,거리_차이,인원수,평균출근,평균퇴근,위경도,poly
0,고양시,40.57,53.62,101.93,27.67,41.55,75.86,-26.07,-12.07,-12.89,29,10:12:25,20:44:35,"(126.831955109648, 37.6584180954929)","[[[126.6703901, 37.6848009], [126.6706681, 37...."
1,과천시,14.14,30.00,57.67,15.97,28.33,51.67,-6.00,-1.67,1.83,3,11:16:21,21:26:57,"(126.987711960677, 37.4292291986772)","[[[127.0113677108152, 37.45540804524003], [127..."
2,광명시,27.07,47.40,69.00,15.84,36.00,54.60,-14.40,-11.40,-11.23,5,10:09:17,20:16:57,"(126.864647815161, 37.4785674780314)","[[[126.86961582865139, 37.49217120215355], [12..."
3,광주시,28.30,37.17,84.67,40.72,55.00,102.00,17.33,17.83,12.42,6,09:55:58,20:09:37,"(127.255138719721, 37.4293941934866)","[[[127.17387456968379, 37.40549980934484], [12..."
4,구리시,18.45,26.80,66.80,24.89,35.40,83.20,16.40,8.60,6.44,5,10:45:30,22:38:02,"(127.129646382723, 37.5943595777621)","[[[127.14396992212652, 37.627711596915304], [1..."


In [43]:
ex_df = aaa[aaa.poly!='none']
ex_df

,지역,MDM_거리2,MDM_자차2,MDM_대중교통2,YTC_거리2,YTC_자차2,YTC_대중교통2,대중교통_차이,자차_차이,거리_차이,인원수,평균출근,평균퇴근,위경도,poly
0,고양시,40.57,53.62,101.93,27.67,41.55,75.86,-26.07,-12.07,-12.89,29,10:12:25,20:44:35,"(126.831955109648, 37.6584180954929)","[[[126.6703901, 37.6848009], [126.6706681, 37...."
1,과천시,14.14,30.00,57.67,15.97,28.33,51.67,-6.00,-1.67,1.83,3,11:16:21,21:26:57,"(126.987711960677, 37.4292291986772)","[[[127.0113677108152, 37.45540804524003], [127..."
2,광명시,27.07,47.40,69.00,15.84,36.00,54.60,-14.40,-11.40,-11.23,5,10:09:17,20:16:57,"(126.864647815161, 37.4785674780314)","[[[126.86961582865139, 37.49217120215355], [12..."
3,광주시,28.30,37.17,84.67,40.72,55.00,102.00,17.33,17.83,12.42,6,09:55:58,20:09:37,"(127.255138719721, 37.4293941934866)","[[[127.17387456968379, 37.40549980934484], [12..."
4,구리시,18.45,26.80,66.80,24.89,35.40,83.20,16.40,8.60,6.44,5,10:45:30,22:38:02,"(127.129646382723, 37.5943595777621)","[[[127.14396992212652, 37.627711596915304], [1..."
5,군포시,26.94,41.25,73.75,28.78,43.75,67.50,-6.25,2.50,1.83,4,09:36:02,20:44:14,"(126.935205932727, 37.3616318289596)","[[[126.88179326587874, 37.3343928309754], [126..."
6,김포시,44.06,53.30,114.00,32.03,40.90,87.50,-26.50,-12.40,-12.03,10,10:02:38,20:13:42,"(126.715657922927, 37.6152686898333)","[[[126.67693566373103, 37.74656983562356], [12..."
7,남양주시,26.29,36.47,81.16,33.61,45.32,94.05,12.89,8.84,7.32,19,10:27:45,21:10:32,"(127.21648030719, 37.6360272346499)","[[[127.26617788371372, 37.78035709680839], [12..."
10,부천시,35.85,57.93,82.40,22.37,42.07,58.93,-23.47,-15.87,-13.48,15,10:15:29,21:34:45,"(126.766038450927, 37.5035117792843)","[[[126.82304049086537, 37.4883803302667], [126..."
11,서울 강남구,3.62,12.02,25.91,11.99,25.64,51.47,25.56,13.62,8.37,91,10:23:44,21:05:39,"(127.047377408384, 37.5173319258532)","[[[127.05621905441494, 37.52832385576861], [12..."


In [44]:
# ex_df.to_excel("dldld.xlsx")

In [45]:
ex_df['polygon'] = tuple(Polygon(i[0]) for i in ex_df['poly'])
ex_df

/Users/jakang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,지역,MDM_거리2,MDM_자차2,MDM_대중교통2,YTC_거리2,YTC_자차2,YTC_대중교통2,대중교통_차이,자차_차이,거리_차이,인원수,평균출근,평균퇴근,위경도,poly,polygon
0,고양시,40.57,53.62,101.93,27.67,41.55,75.86,-26.07,-12.07,-12.89,29,10:12:25,20:44:35,"(126.831955109648, 37.6584180954929)","[[[126.6703901, 37.6848009], [126.6706681, 37....","POLYGON ((126.6703901 37.6848009, 126.6706681 ..."
1,과천시,14.14,30.00,57.67,15.97,28.33,51.67,-6.00,-1.67,1.83,3,11:16:21,21:26:57,"(126.987711960677, 37.4292291986772)","[[[127.0113677108152, 37.45540804524003], [127...","POLYGON ((127.0113677108152 37.45540804524003,..."
2,광명시,27.07,47.40,69.00,15.84,36.00,54.60,-14.40,-11.40,-11.23,5,10:09:17,20:16:57,"(126.864647815161, 37.4785674780314)","[[[126.86961582865139, 37.49217120215355], [12...","POLYGON ((126.8696158286514 37.49217120215355,..."
3,광주시,28.30,37.17,84.67,40.72,55.00,102.00,17.33,17.83,12.42,6,09:55:58,20:09:37,"(127.255138719721, 37.4293941934866)","[[[127.17387456968379, 37.40549980934484], [12...","POLYGON ((127.1738745696838 37.40549980934484,..."
4,구리시,18.45,26.80,66.80,24.89,35.40,83.20,16.40,8.60,6.44,5,10:45:30,22:38:02,"(127.129646382723, 37.5943595777621)","[[[127.14396992212652, 37.627711596915304], [1...","POLYGON ((127.1439699221265 37.6277115969153, ..."
5,군포시,26.94,41.25,73.75,28.78,43.75,67.50,-6.25,2.50,1.83,4,09:36:02,20:44:14,"(126.935205932727, 37.3616318289596)","[[[126.88179326587874, 37.3343928309754], [126...","POLYGON ((126.8817932658787 37.3343928309754, ..."
6,김포시,44.06,53.30,114.00,32.03,40.90,87.50,-26.50,-12.40,-12.03,10,10:02:38,20:13:42,"(126.715657922927, 37.6152686898333)","[[[126.67693566373103, 37.74656983562356], [12...","POLYGON ((126.676935663731 37.74656983562356, ..."
7,남양주시,26.29,36.47,81.16,33.61,45.32,94.05,12.89,8.84,7.32,19,10:27:45,21:10:32,"(127.21648030719, 37.6360272346499)","[[[127.26617788371372, 37.78035709680839], [12...","POLYGON ((127.2661778837137 37.78035709680839,..."
10,부천시,35.85,57.93,82.40,22.37,42.07,58.93,-23.47,-15.87,-13.48,15,10:15:29,21:34:45,"(126.766038450927, 37.5035117792843)","[[[126.82304049086537, 37.4883803302667], [126...","POLYGON ((126.8230404908654 37.4883803302667, ..."
11,서울 강남구,3.62,12.02,25.91,11.99,25.64,51.47,25.56,13.62,8.37,91,10:23:44,21:05:39,"(127.047377408384, 37.5173319258532)","[[[127.05621905441494, 37.52832385576861], [12...","POLYGON ((127.0562190544149 37.52832385576861,..."


In [46]:
# ex_df.to_excel("전처리용.xlsx", index = False, encoding = "utf-8-sig")

In [47]:
# fddor city in aaa.지역:
#     print(the_final['출근'][the_final['지역'] == city].values)

In [48]:
ex_df.columns

Index(['지역', 'MDM_거리2', 'MDM_자차2', 'MDM_대중교통2', 'YTC_거리2', 'YTC_자차2',
       'YTC_대중교통2', '대중교통_차이', '자차_차이', '거리_차이', '인원수', '평균출근', '평균퇴근', '위경도',
       'poly', 'polygon'],
      dtype='object')

In [49]:
# ex_df2 = ex_df[['지역', 'MDM_거리2', 'MDM_자차2', 'MDM_대중교통2', 'YTC_거리2', 'YTC_자차2',
#        'YTC_대중교통2', '대중교통_차이', '자차_차이', '거리_차이', '인원수', '평균출근', '평균퇴근']]
# ex_df2.to_excel("인범님분석용.xlsx", index = False, encoding = "utf-8-sig")

In [50]:
ex_df.head()

,지역,MDM_거리2,MDM_자차2,MDM_대중교통2,YTC_거리2,YTC_자차2,YTC_대중교통2,대중교통_차이,자차_차이,거리_차이,인원수,평균출근,평균퇴근,위경도,poly,polygon
0,고양시,40.57,53.62,101.93,27.67,41.55,75.86,-26.07,-12.07,-12.89,29,10:12:25,20:44:35,"(126.831955109648, 37.6584180954929)","[[[126.6703901, 37.6848009], [126.6706681, 37....","POLYGON ((126.6703901 37.6848009, 126.6706681 ..."
1,과천시,14.14,30.00,57.67,15.97,28.33,51.67,-6.00,-1.67,1.83,3,11:16:21,21:26:57,"(126.987711960677, 37.4292291986772)","[[[127.0113677108152, 37.45540804524003], [127...","POLYGON ((127.0113677108152 37.45540804524003,..."
2,광명시,27.07,47.40,69.00,15.84,36.00,54.60,-14.40,-11.40,-11.23,5,10:09:17,20:16:57,"(126.864647815161, 37.4785674780314)","[[[126.86961582865139, 37.49217120215355], [12...","POLYGON ((126.8696158286514 37.49217120215355,..."
3,광주시,28.30,37.17,84.67,40.72,55.00,102.00,17.33,17.83,12.42,6,09:55:58,20:09:37,"(127.255138719721, 37.4293941934866)","[[[127.17387456968379, 37.40549980934484], [12...","POLYGON ((127.1738745696838 37.40549980934484,..."
4,구리시,18.45,26.80,66.80,24.89,35.40,83.20,16.40,8.60,6.44,5,10:45:30,22:38:02,"(127.129646382723, 37.5943595777621)","[[[127.14396992212652, 37.627711596915304], [1...","POLYGON ((127.1439699221265 37.6277115969153, ..."


## 법인별 특성

In [51]:
the_final.head(2)

,New 사번,재직구분,이름,이메일,파싱용 주소,주소,1-9월\n평균근무시간 (만근월 기준),총OT시간,평균OT시간\n(총OT/OT한 개월수),1-9 OT지급총액,...,거리_차이,출근,퇴근,지역,MDM_거리2,YTC_거리2,YTC_대중교통2,YTC_자차2,MDM_대중교통2,MDM_자차2
0,BHE0502001,재직,방시혁,hitman@bighitcorp.com,서울 강남구 논현동 80-12 404,서울 강남구 논현동 80-12 404,NaN,NaN,NaN,NaN,...,4.03,정보없음,정보없음,서울 강남구,4.58,8.61,43,22,30,14
1,BHE2005004,재직,박지원,jwpark@bighitcorp.com,서울 강남구 선릉로 217 (도곡동) 411동 1004호,서울 강남구 선릉로 217 (도곡동) 411동 1004호,NaN,NaN,NaN,NaN,...,9.64,정보없음,정보없음,서울 강남구,2.55,12.19,52,30,23,9


In [53]:
the_lst = []
for val in the_final['대중교통_차이']:
    if val<0:
        the_val = '감소'
    elif val>0:
        the_val = '증가'
    else:
        the_val = '쌤쌤'
    the_lst.append(the_val)
the_final['교통_증감'] = the_lst


the_lst = []
for val in the_final['자차_차이']:
    if val<0:
        the_val = '감소'
    elif val>0:
        the_val = '증가'
    else:
        the_val = '쌤쌤'
    the_lst.append(the_val)
the_final['자차_증감'] = the_lst

the_lst = []
for val in the_final['거리_차이']:
    if val<0:
        the_val = '감소'
    elif val>0:
        the_val = '증가'
    else:
        the_val = '쌤쌤'
    the_lst.append(the_val)
the_final['거리_증감'] = the_lst


In [54]:
round(the_final['대중교통_차이'].mean(),2)

-1.2

In [55]:
def GET_DATA(the_col):
    if the_col == "교통_증감":
        sub_col = "대중교통_차이"
    elif the_col == "거리_증감":
        sub_col = "거리_차이"
    elif the_col == "자차_증감":
        sub_col = "자차_차이"
        
        
    the_lst = []
    comp_lst = ["전사","BHE",'BHT','BHI','BED','BNX','BLF','SCM','SPB','PLD']
    for the_comp in comp_lst:
        if the_comp == "전사":
            total = the_final[the_col].value_counts().sum()
            check = the_final[the_col].value_counts()
            a = check['증가']
            b = check['감소']
            c = check['쌤쌤']
            avg_up= round(the_final[sub_col][the_final[the_col]=='증가'].mean(),2)
            avg_down= round(the_final[sub_col][the_final[the_col]=='감소'].mean(),2)
            avg_same= round(the_final[sub_col].mean(),2)
        else:
            total = the_final[the_col][the_final['법인'] == the_comp].value_counts().sum()
            check = the_final[the_col][the_final['법인'] == the_comp].value_counts()
            try:
                a = check['증가']
                avg_up = round(the_final[sub_col][(the_final['법인'] == the_comp)&(the_final[the_col]=='증가')].mean(),2)
            except KeyError:
                a = 0
                avg_up = 0
            try:
                b = check['감소']
                avg_down = round(the_final[sub_col][(the_final['법인'] == the_comp)&(the_final[the_col]=='감소')].mean(),2)
            except KeyError:
                b = 0
                avg_down = 0
            try:
                c = check['쌤쌤']
                avg_same = round(the_final[sub_col][the_final['법인'] == the_comp].mean(),2)
            except KeyError:
                c = 0
                avg_same = 0
            
        the_lst.append([the_comp, total, a, b, c, avg_up, avg_down, avg_same])
    pd.DataFrame(columns = ["법인",'총인원','증가','감소','변화없음','증가평균','감소평균','변화평균'], data = the_lst).to_excel("법인별_"+the_col+"_인원수.xlsx", index = False, encoding = "utf-8-sig")
    return pd.DataFrame(columns = ["법인",'총인원','증가','감소','변화없음','증가평균','감소평균','변화평균'], data = the_lst)

In [56]:
akak1 = GET_DATA('교통_증감')
akak1

,법인,총인원,증가,감소,변화없음,증가평균,감소평균,변화평균
0,전사,882,389,479,14,21.41,-19.60,-1.20
1,BHE,352,158,188,6,22.16,-19.40,-0.41
2,BHT,125,53,70,2,23.45,-21.31,-1.99
3,BHI,126,59,65,2,20.61,-18.45,0.13
4,BED,6,2,4,0,26.50,-23.00,0.00
5,BNX,131,56,72,3,18.64,-19.08,-2.52
6,BLF,7,3,4,0,18.67,-24.00,0.00
7,SCM,20,8,12,0,15.88,-19.08,0.00
8,SPB,42,16,26,0,24.12,-19.31,0.00
9,PLD,69,33,35,1,20.73,-19.63,-0.04


In [57]:
# the_num = "79"
# the_res89 = eos_df[GET_QNUM(question_data, the_num)].value_counts(normalize = True)
# ans_lst = ['담당 업무의 특성에 적합하지 않아서','다른 구성원의 눈치가 보여서','명확한 가이드라인이 없어서','제도를 정확히 몰라서','굳이 사용할 필요가 없어서','기타']
# val_lst = the_res89.tolist()[:5] + [sum(the_res89.tolist()[5:])]
# colors = ['yellowgreen', 'beige', 'lightgreen', 'lightskyblue', 'lightgrey', 'lightpink']

# plt.figure(figsize = (7, 7))
# plt.pie(x = val_lst, labels = ans_lst, autopct='%.1f%%', startangle=90, colors = colors, explode = [0.02 for i in range(0, len(ans_lst))])
# plt.savefig("img/Q"+the_num+".png", bbox_inches = "tight")
# plt.show()

# print("*기타 의견:")
# skip_lst = ["없음",'없습니다',".",'0','없다']
# for i in the_res89.index:
#     if i not in ans_lst:
#         if i not in skip_lst:
#             print(i.strip())

In [58]:
GET_DATA('거리_증감')

,법인,총인원,증가,감소,변화없음,증가평균,감소평균,변화평균
0,전사,882,470,411,1,7.22,-10.42,-1.01
1,BHE,352,194,157,1,7.16,-10.14,-0.58
2,BHT,125,58,67,0,8.83,-11.18,0.00
3,BHI,126,73,53,0,7.21,-10.71,0.00
4,BED,6,2,4,0,9.86,-10.63,0.00
5,BNX,131,69,62,0,6.41,-10.12,0.00
6,BLF,7,3,4,0,5.11,-12.26,0.00
7,SCM,20,9,11,0,6.91,-9.99,0.00
8,SPB,42,22,20,0,6.72,-11.07,0.00
9,PLD,69,39,30,0,7.08,-9.93,0.00


In [59]:
GET_DATA('자차_증감')

,법인,총인원,증가,감소,변화없음,증가평균,감소평균,변화평균
0,전사,882,409,455,18,11.04,-13.55,-1.87
1,BHE,352,184,160,8,12.51,-11.71,1.22
2,BHT,125,56,68,1,13.93,-15.46,-2.17
3,BHI,126,64,56,6,9.98,-14.02,-1.16
4,BED,6,2,4,0,7.00,-16.50,0.00
5,BNX,131,51,80,0,8.59,-13.79,0.00
6,BLF,7,1,6,0,4.00,-12.83,0.00
7,SCM,20,7,13,0,3.29,-15.15,0.00
8,SPB,42,14,28,0,7.00,-14.36,0.00
9,PLD,69,28,38,3,7.50,-15.16,-5.30


In [60]:
# pd.DataFrame(columns = ["법인",'총인원','증가','감소','변화없음'], data = the_lst).to_excel("법인별_증감_인원수.xlsx", index = False, encoding = "utf-8-sig")

In [135]:
def GET_GROUP(the_map, the_col, group, color):
    the_group = folium.FeatureGroup(name=str(group)).add_to(the_map)
    for val in the_final[['위도','경도']][the_final[str(the_col)] == str(group)].values:
        val = tuple(val)
        the_group.add_child(folium.Circle(val, radius = 4, color = str(color)))

In [138]:
comp_lst = ['BHE','BHT','BHI','BED','BNX','BLF','SCM','SPB','PLD']
color_lst = ['green','orange','yellow','black','purple','grey','red','blue','brown']

In [139]:
YTC = ['37.5245456464592', '126.964197674537']
MDM = ['37.5068182254549', '127.066242271781']

the_map= folium.Map(YTC, zoom_start=10, tiles='cartodbpositron')
folium.Marker(location=YTC,  tooltip = "용산트레이드센터", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)
folium.Marker(location=MDM,  tooltip = "엠디엠타워", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)


for comp, color in zip(comp_lst, color_lst):
    GET_GROUP(the_map, "법인", comp, color)
folium.LayerControl().add_to(the_map)
the_map

# =============== 연령대 ===============

In [61]:
val_lst = []
for i in the_final['age']:
    if i >=20 and i<30:
        val = '20대'
    elif i >= 30 and i <40:
        val = '30대'
    elif i >= 40 and i<50:
        val = '40이상'
    val_lst.append(val)
the_final['연령대'] = val_lst

In [62]:
the_final['연령대'].value_counts()

30대     485
20대     301
40이상     96
Name: 연령대, dtype: int64

In [78]:
val_lst = []
for i in the_final['age']:
    if i >=20 and i<25:
        val = '20초중'
    elif i>=25 and i<30:
        val = "20중후"
    elif i >= 30 and i <35:
        val = '30초중'
    elif i>=35 and i<40:
        val = "30중후"
    elif i >= 40:
        val = '40이상'
    val_lst.append(val)
the_final['연령세분'] = val_lst

In [79]:
df_age = pd.DataFrame(the_final['연령세분'].value_counts()).reset_index().rename({"연령세분":"cnt", "index":"연령대2"}, axis = 1)
df_age

,연령대2,cnt
0,30초중,279
1,20중후,269
2,30중후,204
3,40이상,99
4,20초중,31


In [80]:
val_lst = []
for age in df_age['연령대2'].values:
    val = meanTime([i for i in the_final['출근'][the_final['연령세분']==age].values if i !='정보없음'])
    val_lst.append(val)
df_age['출근시간'] = val_lst

val_lst = []
for age in df_age['연령대2'].values:
    val = meanTime([i for i in the_final['퇴근'][the_final['연령세분']==age].values if i !='정보없음'])
    val_lst.append(val)
df_age['퇴근시간'] = val_lst
df_age.sort_values('연령대2',ascending = False)

,연령대2,cnt,출근시간,퇴근시간
3,40이상,99,09:53:31,20:49:34
0,30초중,279,10:19:34,20:54:12
2,30중후,204,10:13:01,20:53:17
4,20초중,31,10:43:24,22:09:38
1,20중후,269,10:38:04,21:14:41


In [81]:
thus_lst = []
for i in df_age['출근시간']:
    hour = str(int(i.split(":")[0]))
    minn = str(int(i.split(":")[1]))
    thus = float(hour+"."+minn)
    thus_lst.append(thus)
df_age['출근'] = thus_lst

thus_lst = []
for i in df_age['퇴근시간']:
    hour = str(int(i.split(":")[0]))
    minn = str(int(i.split(":")[1]))
    thus = float(hour+"."+minn)
    thus_lst.append(thus)
df_age['퇴근'] = thus_lst
df_age


,연령대2,cnt,출근시간,퇴근시간,출근,퇴근
0,30초중,279,10:19:34,20:54:12,10.19,20.54
1,20중후,269,10:38:04,21:14:41,10.38,21.14
2,30중후,204,10:13:01,20:53:17,10.13,20.53
3,40이상,99,09:53:31,20:49:34,9.53,20.49
4,20초중,31,10:43:24,22:09:38,10.43,22.90


In [123]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

start = '10:13:00'
end = '20:53:00' # for example
start_dt = datetime.strptime(start, "%H:%M:%S")
end_dt = datetime.strptime(end, "%H:%M:%S")
timedelta_obj = relativedelta(end_dt, start_dt)
print(
    timedelta_obj.hours,
    timedelta_obj.minutes,
    timedelta_obj.seconds,
)

10 40 0


In [83]:
def CREATE_TITLE(the_df):
    lst = []
    for i in the_df.직책:
        if (i == '팀원') or (i == 'LP') or (i == 'Project Lead') or (i == 'SP'):
            val = '팀원'
            lst.append(val)
        elif (i == 'Chief') or (i == '사업_CEO') or (i == 'President'):
            val = 'Chief'
            lst.append(val)
        elif (i == 'Head of Business') or (i == 'VP'):
            val = 'VP'
            lst.append(val)
        elif '팀장' in i:
            val = '팀장'
            lst.append(val)
        elif '파트장' in i:
            val = '파트장'
            lst.append(val)
        else:
            val = i
            lst.append(val)
    the_final['직책통합'] = lst
    return the_final

In [84]:
the_final = CREATE_TITLE(the_final)
the_final

,New 사번,재직구분,이름,이메일,파싱용 주소,주소,1-9월\n평균근무시간 (만근월 기준),총OT시간,평균OT시간\n(총OT/OT한 개월수),1-9 OT지급총액,...,YTC_대중교통2,YTC_자차2,MDM_대중교통2,MDM_자차2,교통_증감,자차_증감,거리_증감,연령대,연령세분,직책통합
0,BHE0502001,재직,방시혁,hitman@bighitcorp.com,서울 강남구 논현동 80-12 404,서울 강남구 논현동 80-12 404,NaN,NaN,NaN,NaN,...,43,22,30,14,증가,증가,증가,40이상,40이상,의장
1,BHE2005004,재직,박지원,jwpark@bighitcorp.com,서울 강남구 선릉로 217 (도곡동) 411동 1004호,서울 강남구 선릉로 217 (도곡동) 411동 1004호,NaN,NaN,NaN,NaN,...,52,30,23,9,증가,증가,증가,40이상,40이상,HQ CEO
2,BHE1903010,재직,김태호,ian@bighitcorp.com,서울 강남구 언주로 626 로얄팰리스 1201호,서울 강남구 언주로 626 로얄팰리스 1201호,NaN,NaN,NaN,NaN,...,43,22,29,12,증가,증가,증가,40이상,40이상,Chief
3,BHE1801001,재직,이진형,aj@bighitcorp.com,경기도 성남시 수정구 위례동로 15,경기도 성남시 수정구 위례동로 15,NaN,NaN,NaN,NaN,...,91,41,54,20,증가,증가,증가,40이상,40이상,Chief
4,BHE0807001,재직,김신규,bighit4832@bighitcorp.com,서울시 서초구 잠원로117 \n아크로리버뷰 104동2204호,서울시 서초구 잠원로117 \n아크로리버뷰 104동2204호,NaN,NaN,NaN,NaN,...,37,18,38,21,감소,감소,감소,40이상,40이상,Chief
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,PLD1911005,재직,박지혜B,jhpark@pledis.co.kr,서울시 종로구 소격동 35-2,서울시 종로구 소격동 35-2,NaN,NaN,NaN,NaN,...,37,23,61,36,감소,감소,감소,30대,30중후,팀원
878,PLD1808002,재직,가하,kahastar@pledis.co.kr,서울시 양천구 신정동 944-4,서울시 양천구 신정동 944-4,NaN,NaN,NaN,NaN,...,46,21,70,44,감소,감소,감소,30대,30초중,팀원
879,PLD1707001,재직,강다은B,kangda93@pledis.co.kr,경기도 의왕시 포일동 이미로 17 포일숲속마을,경기도 의왕시 포일동 이미로 17. 포일숲속마을 2단지 210동 603호,NaN,NaN,NaN,NaN,...,63,37,69,37,감소,쌤쌤,감소,20대,20중후,팀원
880,PLD1801001,재직,정다예,dyjeong@pledis.co.kr,서울시 동작구 대방동411-33 팰리스타운 602호,서울시 동작구 대방동411-33 팰리스타운 602호,NaN,NaN,NaN,NaN,...,29,14,55,35,감소,감소,감소,20대,20중후,팀원


In [85]:
the_final[['연령세분','직책통합','이름']].groupby(['연령세분','직책통합']).count().reset_index()

,연령세분,직책통합,이름
0,20중후,Professional,1
1,20중후,팀원,265
2,20중후,팀장,1
3,20중후,파트장,2
4,20초중,팀원,30
5,20초중,팀장,1
6,30중후,Chief,1
7,30중후,Head of X,1
8,30중후,Professional,1
9,30중후,VP,1


In [120]:
ff = the_final[['연령세분','실','이름']].groupby(['연령세분','실']).count().reset_index()
ff[ff['연령세분']=='20초중']

,연령세분,실,이름
21,20초중,360사업실,1
22,20초중,Creative Studios,2
23,20초중,R&D Center,1
24,20초중,리테일사업실,1
25,20초중,사업기획실,1
26,20초중,아티스트매니지먼트실,1
27,20초중,음악제작실,3
28,20초중,재무실,1
29,20초중,제작실,1
30,20초중,커뮤니케이션실,2


In [115]:
dd1 = the_final[['연령세분','1-9 OT수당평균\n(총지급액/지급개월수)']].dropna().groupby(['연령세분']).count().reset_index()
dd2 = the_final[['연령세분','1-9 OT수당평균\n(총지급액/지급개월수)']].dropna().groupby(['연령세분']).sum().reset_index()
dd3 = pd.merge(dd1, dd2, on = "연령세분", how = "left" )
dd3['diff'] = dd3['1-9 OT수당평균\n(총지급액/지급개월수)_y'] / dd3['1-9 OT수당평균\n(총지급액/지급개월수)_x']
dd3

,연령세분,1-9 OT수당평균\n(총지급액/지급개월수)_x,1-9 OT수당평균\n(총지급액/지급개월수)_y,diff
0,20중후,206,4.370132e+07,212142.311071
1,20초중,16,9.057820e+06,566113.736111
2,30중후,166,4.111493e+07,247680.321060
3,30초중,225,7.993956e+07,355286.951160
4,40이상,69,2.303509e+07,333841.853865


In [111]:
4.370132e+07/205

213177.17073170733

In [125]:
dd2 = the_final[['연령세분','1-9월\n평균근무시간 (만근월 기준)']].dropna().groupby(['연령세분']).mean().reset_index()
dd2

,연령세분,1-9월\n평균근무시간 (만근월 기준)
0,20중후,188.544443
1,20초중,226.604456
2,30중후,180.323682
3,30초중,189.549326
4,40이상,186.271264


In [94]:
the_final.columns

Index([                  'New 사번',                     '재직구분',
                             '이름',                      '이메일',
                         '파싱용 주소',                       '주소',
          '1-9월\n평균근무시간 (만근월 기준)',                    '총OT시간',
          '평균OT시간\n(총OT/OT한 개월수)',               '1-9 OT지급총액',
       '1-9 OT수당평균\n(총지급액/지급개월수)',                      'age',
                             '성별',                       '법인',
                        '직속\n직책자',                    '담당 리더',
             '최종\nReporting Line',                       '부문',
                             '본부',                        '실',
                              '팀',                       '파트',
                             '담당',                       '직책',
                          '겸직 여부',                       '겸직',
                             '유형',                     '입사경로',
                          '그룹입사일',                     '입사일자',
                           '입사구분',                   'n

In [76]:
the_final[(the_final.직책통합== 'GM') & (the_final.연령세분 =='20중후')]

,New 사번,재직구분,이름,이메일,파싱용 주소,주소,1-9월\n평균근무시간 (만근월 기준),총OT시간,평균OT시간\n(총OT/OT한 개월수),1-9 OT지급총액,...,YTC_대중교통2,YTC_자차2,MDM_대중교통2,MDM_자차2,교통_증감,자차_증감,거리_증감,연령대,연령세분,직책통합
481,BHI1910069,재직,이승석,ispano@bighitcorp.com,"서울 중구 다산로 32 (신당동, 남산타운) 2동 506호","서울 중구 다산로 32 (신당동, 남산타운) 2동 506호",NaN,NaN,NaN,NaN,...,39,25,50,25,감소,쌤쌤,감소,20대,20중후,GM


In [77]:
the_final['age'][the_final.이름 == '이승석']

481    51.0
Name: age, dtype: float64

In [126]:
YTC = ['37.5245456464592', '126.964197674537']

the_map= folium.Map(YTC, zoom_start=10, tiles='cartodbpositron')
group1 = folium.FeatureGroup(name="20초중반").add_to(the_map)
for val in the_final[['위도','경도']][the_final['연령세분'] == '20초중'].values:
    val = tuple(val)
#     icon = folium.Icon(icon_color="blue")
    group1.add_child(folium.Circle(val, radius = 2, color = "blue"))
group2 = folium.FeatureGroup(name="20중후반").add_to(the_map)
for val in the_final[['위도','경도']][the_final['연령세분'] == '20중후'].values:
    val = tuple(val)
#     icon = folium.Icon(icon_color="red")
#     group2.add_child(folium.Circle(val, icon = icon))
    group2.add_child(folium.Circle(val, radius = 2, color = "red"))
group3 = folium.FeatureGroup(name="30초중반").add_to(the_map)
for val in the_final[['위도','경도']][the_final['연령세분'] == '30초중'].values:
    val = tuple(val)
#     icon = folium.Icon(icon_color="yellow")
#     group3.add_child(folium.Circle(val, icon = icon))
    group3.add_child(folium.Circle(val, radius = 2, color = "yellow"))
group4 = folium.FeatureGroup(name="30중후반").add_to(the_map)
for val in the_final[['위도','경도']][the_final['연령세분'] == '30중후'].values:
    val = tuple(val)
#     icon = folium.Icon(icon_color="grey")
#     group4.add_child(folium.Circle(val, icon = icon))
    group4.add_child(folium.Circle(val, radius = 2, color = "grey"))
group5 = folium.FeatureGroup(name="40대이상").add_to(the_map)
for val in the_final[['위도','경도']][the_final['연령세분'] == '40이상'].values:
    val = tuple(val)
#     icon = folium.Icon(icon_color="white")
#     group5.add_child(folium.Circle(val, icon = icon))
    group5.add_child(folium.Circle(val, radius = 2, color = "black"))

folium.LayerControl().add_to(the_map)
the_map

### =============== 환승역 ===============

In [ ]:
sub = '역'
for sub_lst in the_mdm[['대중교통_환승1','대중교통_환승2','대중교통_환승3','대중교통_환승4']].values.tolist():
    for text in sub_lst:
        if sub in text:
            print(text)

In [ ]:
def FIND_STATIONS(hi):
    the_lst = []
    try:
        new_lst = [re.sub('[^\w\s]',' ',i).split(" ") for i in hi]
    except TypeError:
        print("!!!!!!!!!", hi)
        new_lst = hi
        pass
    for lst in new_lst:
        try:
            for i in lst:
                if "역" in i:
                    the_lst.append(re.sub(" ","",i))
        except TypeError:
            pass
    return the_lst

In [ ]:
station_lst = []
for who, lst in zip(the_ytc['이름'], the_ytc[['대중교통_환승1','대중교통_환승2','대중교통_환승3','대중교통_환승4']].values.tolist()):
    try:
        uni_lst = list(set(FIND_STATIONS(lst)))
    except TypeError:
        print("!!!!!!!!!!", FIND_STATIONS(lst))
#     print(who, uni_lst)
    station_lst.append(uni_lst)
station_lst = sum(station_lst, [])
station_lst = [i.split("역")[0]+"역" for i in station_lst if i != '역']
# station_lst

In [ ]:
the_station = []
for i in station_lst:
    if i != '역' and i!="동북아역":
        if "호선" in i:
            val = i.split("호선")[1]
        else:
            val = i
        the_station.append(val)
# the_station

In [ ]:
df_subway = pd.DataFrame(columns = ["역"], data = the_station)
df_sub = pd.DataFrame(df_subway['역'].value_counts()).reset_index().rename({"역":"cnt", "index":"station"}, axis = 1)
# df_sub = df_sub.rename({"역":"cnt", "index":"station"})
df_sub

In [ ]:
val_lst = []
for i in df_sub['station']:
    print(i)
    val = GET_KAKAO_GEO(i)
    val_lst.append(val)
df_sub['위경도'] = val_lst
df_sub

In [ ]:
def GET_THE_MAP3(the_df, who_lst, filename):
    
    YTC = ['37.5245456464592', '126.964197674537']
    MDM = ['37.5068182254549', '127.066242271781']
#     {'darkred', 'beige', 'darkpurple', 'darkblue', 'cadetblue', 'darkgreen', 'blue', 'black', 'lightgreen', 'lightblue', 'white', 'gray', 'lightgray', 'red', 'green', 'pink', 'orange', 'lightred', 'purple'}.
    the_map = folium.Map(location=YTC, tiles='cartodbpositron', zoom_start=10)
#     folium.GeoJson(polygon).add_to(the_map)

    folium.Marker(location=YTC,  tooltip = "용산트레이드센터", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)
    
    folium.Marker(location=MDM,  tooltip = "엠디엠타워", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)
    
    for km in [10000,20000,30000,40000,50000]:

        folium.Circle(location=YTC,
                      tooltip = str(km)[:2]+"km (직선거리)",
                      radius= km, 
                      color = "red", 
                      weight = 1
                     ).add_to(the_map)

    feature_group = folium.FeatureGroup("Locations")
#     the_loc_lst = []
    for who in who_lst:
        the_who = who
        the_cnt = str(the_df['cnt'][the_df['station'] == who].values[0])
#         the_who = emoji.emojize(":bust_in_silhouette: "+ who)
#         pub_avg = emoji.emojize("<br>:railway_car: "+ " MDM "+the_df['MDM_대중교통'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_대중교통'][the_df.이름 == str(who)].values[0])
#         self_avg =  emoji.emojize("<br>:automobile: "+ " MDM "+the_df['MDM_자차'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_자차'][the_df.이름 == str(who)].values[0])
#         dist_avg = emoji.emojize("<br>:hourglass_not_done:"+" MDM "+the_df['MDM_거리'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_거리'][the_df.이름 == str(who)].values[0])
#         the_in = emoji.emojize("<br>:sunrise: "+ the_df['출근'][the_df.이름 == str(who)].values[0])
#         the_out = emoji.emojize("<br>:bridge_at_night: "+ the_df['퇴근'][the_df.이름 == str(who)].values[0])
        the_label = the_who+"\n"+the_cnt#str(the_who) + pub_avg + self_avg + dist_avg + the_in + the_out

        the_loc_lst = list(the_df[['위경도']][the_df['station'] == str(who)].values[0].tolist()[0])
        the_loc = [the_loc_lst[1], the_loc_lst[0]]
        iframe = folium.IFrame(the_label, width=300, height=200)
        popup = folium.Popup(iframe, max_width=300)
        if int(the_cnt) >=100:
            the_cnt = math.log(int(the_cnt))+30
        elif int(the_cnt) >= 50 and int(the_cnt) <100:
            the_cnt = math.log(int(the_cnt))+10
        elif int(the_cnt) >= 10 and int(the_cnt) <50:
            the_cnt = math.log(int(the_cnt))+5
        else:
            the_cnt = 1
        folium.CircleMarker(the_loc, radius = int(the_cnt), color='red', fill_color='#3186cc', popup = popup).add_to(the_map)
#     folium.TileLayer('Mapbox Bright').add_to(the_map)
#     folium.TileLayer('Stamen Toner').add_to(the_map)
#     folium.TileLayer('stamentoner').add_to(the_map)
#     folium.TileLayer('openstreetmap').add_to(the_map)
#         the_loc_lst.append(the_loc)
    
    # red, grey, lightgreen, brown, pink(violet), blue, gold,
#     the_map= add_categorical_legend(the_map, '거리 가감구분 (구성원수)',
#                                     colors = ['#FF0000','#A9A9A9','#90EE90',
#                                               '#D2691E','#EE82EE','#0000FF','#FFD700'],
#                                     labels = ['10km 이상 증가(122명)', '5~10km 증가(215명)', '0~5km 증가(133명)',
#                                               '0~5km 감소(52명)', '5~10km 감소(75명)','10~15km 감소(271명)','15km 이상 감소 (14명)'])
    
#     the_map.save("*MAP_"+filename+'.html')
    return the_map

In [ ]:
import math
math.log(100)+30, math.log(50)+10, math.log(9)+1

In [ ]:
who_lst = df_sub.station.tolist()
GET_THE_MAP3(df_sub, who_lst, 'dd')

In [ ]:
GET_THE_MAP(df_poly, '대중교통_증감', 'bluered')

In [ ]:
df_sub.cnt.describe()

In [ ]:
df_sub.for lst in the_mdm[['대중교통_환승1','대중교통_환승2','대중교통_환승3','대중교통_환승4']].values:
    print

## =============== 지역별 ===============

In [153]:
the_final.columns

Index([                  'New 사번',                     '재직구분',
                             '이름',                      '이메일',
                         '파싱용 주소',                       '주소',
          '1-9월\n평균근무시간 (만근월 기준)',                    '총OT시간',
          '평균OT시간\n(총OT/OT한 개월수)',               '1-9 OT지급총액',
       '1-9 OT수당평균\n(총지급액/지급개월수)',                      'age',
                             '성별',                       '법인',
                        '직속\n직책자',                    '담당 리더',
             '최종\nReporting Line',                       '부문',
                             '본부',                        '실',
                              '팀',                       '파트',
                             '담당',                       '직책',
                          '겸직 여부',                       '겸직',
                             '유형',                     '입사경로',
                          '그룹입사일',                     '입사일자',
                           '입사구분',                   'n

### 증가

In [165]:
the_final['지역'][the_final['거리_차이'] >=1].value_counts().sum()

440

In [166]:
the_final['지역'][the_final['대중교통_차이'] >=1].value_counts().sum()

389

### 감소

In [161]:
the_final['지역'][the_final['거리_차이'] <=-1].value_counts()

서울 마포구      50
서울 관악구      47
서울 용산구      33
서울 동작구      30
고양시         29
서울 영등포구     27
서울 강서구      21
인천          20
서울 은평구      17
부천시         15
서울 구로구      14
서울 양천구      13
서울 서대문구     10
김포시         10
서울 서초구       9
서울 중구        9
서울 종로구       7
서울 성북구       7
파주시          5
서울 성동구       5
광명시          5
서울 금천구       5
시흥시          3
안양시          3
안산시          2
수원시          1
의왕시          1
의정부시         1
서울시 서대문구     1
Name: 지역, dtype: int64

In [164]:
the_final['지역'][the_final['대중교통_차이'] <=-1].value_counts().sum()

479

### 차이 없음

In [158]:
the_final['지역'][the_final['거리_차이'] ].value_counts()

서울 강남구    4
성남 분당구    2
서울 송파구    1
성남 수정구    1
Name: 지역, dtype: int64

In [157]:
the_final['지역'][(the_final['대중교통_차이'] >-1)&(the_final['대중교통_차이'] <1)].value_counts()

서울 성동구     3
서울 관악구     2
서울 노원구     1
부산광역시      1
서울 동대문구    1
대전광역시      1
서울 서초구     1
울산광역시      1
서울 서대문구    1
서울 송파구     1
서울 용산구     1
Name: 지역, dtype: int64

In [141]:
# ex_df['the_poly'] = 
crs = {'init': 'epsg:4326'}
df_poly = gpd.GeoDataFrame(crs=crs, geometry=ex_df['polygon'])       
df_poly['지역'] = ex_df['지역']
df_poly['인원수'] = ex_df['인원수']
df_poly['대중교통_증감'] = ex_df['대중교통_차이']
df_poly['거리차이'] = ex_df['거리_차이']
df_poly['평균출근'] = ex_df['평균출근']
df_poly['평균퇴근'] = ex_df['평균퇴근']
df_poly['YTC_소요시간'] = ex_df['YTC_대중교통2']

/Users/jakang/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [144]:
df_poly['지역'][(df_poly.대중교통_증감 >-1)&(df_poly.대중교통_증감<1)]

Series([], Name: 지역, dtype: object)

In [143]:
df_poly['지역'][(df_poly.거리차이 >-1)&(df_poly.거리차이<1)]

44    안양시
Name: 지역, dtype: object

In [ ]:
# def GET_THE_MAP3(the_df, who_lst, filename):
    
#     YTC = ['37.5245456464592', '126.964197674537']
#     MDM = ['37.5068182254549', '127.066242271781']
# #     {'darkred', 'beige', 'darkpurple', 'darkblue', 'cadetblue', 'darkgreen', 'blue', 'black', 'lightgreen', 'lightblue', 'white', 'gray', 'lightgray', 'red', 'green', 'pink', 'orange', 'lightred', 'purple'}.
#     the_map = folium.Map(location=YTC, tiles='cartodbpositron', zoom_start=10)
#     folium.GeoJson(polygon).add_to(the_map)

#     folium.Marker(location=YTC,  tooltip = "용산트레이드센터", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)
    
#     folium.Marker(location=MDM,  tooltip = "엠디엠타워", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)
    
#     for km in [10000,20000,30000,40000,50000]:

#         folium.Circle(location=YTC,
#                       tooltip = str(km)[:2]+"km (직선거리)",
#                       radius= km, 
#                       color = "red", 
#                       weight = 1
#                      ).add_to(the_map)

#     feature_group = folium.FeatureGroup("Locations")
# #     the_loc_lst = []
#     for who in who_lst:
#         the_who = who
#         the_cnt = str(the_df['인원수'][the_df['지역'] == who].values[0])
# #         the_who = emoji.emojize(":bust_in_silhouette: "+ who)
# #         pub_avg = emoji.emojize("<br>:railway_car: "+ " MDM "+the_df['MDM_대중교통'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_대중교통'][the_df.이름 == str(who)].values[0])
# #         self_avg =  emoji.emojize("<br>:automobile: "+ " MDM "+the_df['MDM_자차'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_자차'][the_df.이름 == str(who)].values[0])
# #         dist_avg = emoji.emojize("<br>:hourglass_not_done:"+" MDM "+the_df['MDM_거리'][the_df.이름 == str(who)].values[0] + " / YTC " +  the_df['YTC_거리'][the_df.이름 == str(who)].values[0])
# #         the_in = emoji.emojize("<br>:sunrise: "+ the_df['출근'][the_df.이름 == str(who)].values[0])
# #         the_out = emoji.emojize("<br>:bridge_at_night: "+ the_df['퇴근'][the_df.이름 == str(who)].values[0])
#         the_label = the_who+"\n"+the_cnt#str(the_who) + pub_avg + self_avg + dist_avg + the_in + the_out

#         the_loc_lst = list(the_df[['위경도']][the_df['지역'] == str(who)].values[0].tolist()[0])
#         the_loc = [the_loc_lst[1], the_loc_lst[0]]
#         iframe = folium.IFrame(the_label, width=300, height=200)
#         popup = folium.Popup(iframe, max_width=300)
        
#         folium.CircleMarker(the_loc, radius = int(the_cnt)*0.3, color='#3186cc', fill_color='#3186cc', popup = popup).add_to(the_map)
        
# #         the_loc_lst.append(the_loc)
    
#     # red, grey, lightgreen, brown, pink(violet), blue, gold,
# #     the_map= add_categorical_legend(the_map, '거리 가감구분 (구성원수)',
# #                                     colors = ['#FF0000','#A9A9A9','#90EE90',
# #                                               '#D2691E','#EE82EE','#0000FF','#FFD700'],
# #                                     labels = ['10km 이상 증가(122명)', '5~10km 증가(215명)', '0~5km 증가(133명)',
# #                                               '0~5km 감소(52명)', '5~10km 감소(75명)','10~15km 감소(271명)','15km 이상 감소 (14명)'])
    
# #     the_map.save("*MAP_"+filename+'.html')
#     return the_map

In [ ]:
# area_lst = [i for i in aaa.지역]
# # 
# GET_THE_MAPㅇㅇ3(aaa, area_lst,'dd')

In [ ]:
# df_poly['대중교통'].iloc[0]

In [ ]:
# pd.DataFrame(df_poly.iloc[0]).T

In [ ]:
# map_dict1 = df_poly.set_index("지역")['인원수'].to_dict()
# map_dict2 = df_poly.set_index("지역")['대중교통'].to_dict()


In [ ]:
# from branca.colormap import LinearColormap
# import geopandas as gpd

# color_scale1 = LinearColormap(['lightgreen','green'], vmin = min(map_dict1.values()), vmax = max(map_dict1.values()))
# color_scale2 = LinearColormap(['skyblue','red'], vmin = min(map_dict2.values()), vmax = max(map_dict2.values()))


In [ ]:
# # YTC = [37.5245456464592, 126.964197674537]

# # the_map= folium.Map(YTC, zoom_start=9, tiles='cartodbpositron')
# the_map = folium.plugins.DualMap(YTC, zoom_start = 9)

# style_function1= lambda feature: {
#         'fillColor': get_color(feature, map_dict1,color_scale1),
#         'fillOpacity': 0.6,
#         'color' : 'black',
#         'weight' : 1,
#     }    

# the_map1 = folium.GeoJson(df_poly, style_function = style_function1, tooltip = folium.GeoJsonTooltip(fields = ['지역','인원수'])).add_to(the_map.m1)
# # folium.LayerControl(collapsed = False).add_to(the_map.m1)
# color_scale1.caption = "지역 별 구성원 수"
# color_scale1.add_to(the_map.m1)

# # the_map2= folium.Map(YTC, zoom_start=9, tiles='cartodbpositron')
# style_function2 = lambda feature: {
#         'fillColor': get_color(feature, map_dict2,color_scale2),
#         'fillOpacity': 0.6,
#         'color' : 'black',
#         'weight' : 1,
#     }    

# the_map2 = folium.GeoJson(df_poly, style_function = style_function2, tooltip = folium.GeoJsonTooltip(fields = ['지역','대중교통','인원수'])).add_to(the_map.m2)
# color_scale2.caption = "지역 별 대중교통 증감시간"
# color_scale2.add_to(the_map.m2)

# # folium.plugins.MousePosition().add_to(the_map)
# folium.LayerControl(collapsed=False).add_to(m)
# folium.TileLayer("cartodbpositron").add_to(the_map.m1)
# folium.TileLayer("cartodbpositron").add_to(the_map.m2)


# # the_map1.geojson.add_child(folium.features.G)
# the_map.get_root().html.add_child(folium.Element())

# # the_map1.geojson.add_child(folium.feature)
# the_map

In [ ]:
df_poly.head()

In [173]:
def get_color(feature, map_dict, color_scale):
    value = map_dict.get(feature['properties']['지역'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)
    
def GET_THE_MAP(df_poly, the_col, the_color):
    if the_col == '인원수':
        map_dict = df_poly.set_index("지역")[the_col].to_dict()
        the_text = "지역 별 구성원 수"
    elif the_col == '대중교통_증감':
        map_dict = df_poly.set_index("지역")[the_col].to_dict()
        the_text = '지역 별 통근시간'  
        
    if the_color == 'green':
        color_scale = LinearColormap(['lightgreen','green'], vmin = min(map_dict.values()), vmax = max(map_dict.values()))
    elif the_color == 'bluered':
        color_scale = LinearColormap(['skyblue','red'], vmin = min(map_dict.values()), vmax = max(map_dict.values()))
    
    YTC = ['37.5245456464592', '126.964197674537']
    MDM = ['37.5068182254549', '127.066242271781']

    the_map= folium.Map(YTC, zoom_start=10, tiles='cartodbpositron')
    folium.Marker(location=YTC,  tooltip = "용산트레이드센터", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)

    folium.Marker(location=MDM,  tooltip = "엠디엠타워", icon = folium.Icon(icon = "star", color = "red")).add_to(the_map)

    for km in [10000,20000,30000,40000,50000]:

        folium.Circle(location=YTC,
                      tooltip = str(km)[:2]+"km (직선거리)",
                      radius= km, 
                      color = "red", 
                      weight = 1
                     ).add_to(the_map)

    style_function1= lambda feature: {
            'fillColor': get_color(feature, map_dict, color_scale),
            'fillOpacity': 0.6,
            'color' : 'black',
            'weight' : 1,
        }    

    folium.GeoJson(df_poly, style_function = style_function1, label = '인원수',tooltip = folium.GeoJsonTooltip(fields = ['지역','인원수','YTC_소요시간','대중교통_증감','평균출근','평균퇴근'])).add_to(the_map)
    # folium.LayerControl(collapsed = False).add_to(the_map.m1)
    color_scale.caption = the_text
    color_scale.add_to(the_map)

    # folium.plugins.MousePosition().add_to(the_map)
#     folium.LayerControl(collapsed=False).add_to(the_map)
    
    the_map.save("MAP_"+the_col+'.html')
    return the_map

In [174]:
GET_THE_MAP(df_poly, '인원수', 'green')

TypeError: __init__() got an unexpected keyword argument 'label'

In [172]:
GET_THE_MAP(df_poly, '대중교통_증감', 'bluered')

In [ ]:
aaaaaaa

In [ ]:
df_poly.head()

In [ ]:
# df_poly[['인원수','지역','대중교통']][df_poly.평균출근>'10:00:00']

In [ ]:
# df_poly[['인원수','지역','대중교통']][df_poly.평균출근>'10:00:00']

In [ ]:
df_poly

In [ ]:
m = folium.Map([43, -100], zoom_start=4)

choropleth = folium.Choropleth(
    geo_data=df_poly,
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    name="Unenployment",
    show=False,
).add_to(m)

# The underlying GeoJson and StepColormap objects are reachable
print(type(choropleth.geojson))
print(type(choropleth.color_scale))

folium.LayerControl(collapsed=False).add_to(m)

m

In [ ]:
def json_to_geojson(data, districts):
    # create a geojson from a list of dictionaries
    # containing coordinates with the name of the polygon
    # in our case a polygon is a district
    assert type(data) == list, "The parameter data should be a list of coordinates with a name argument!"
    
    geojson = {
        "type": "FeatureCollection",
        "features": [
        {
            "type": "Feature",
            "geometry" : {
                "type": "Polygon",
                "name": district,
                "coordinates": [[[d["lon"], d["lat"]] for d in data if d['name'] == district]],
                },
            "properties" : {'name': district},
            
         } for district in districts]
    }
    
    return geojson

In [ ]:
aa = sum(ex_df['poly'][ex_df['지역'] == '김포시'].tolist()[0],[])
aa

In [ ]:

geojson = json_to_geojson(aa, "김포시")
geojson

In [ ]:
# from polygon_geohasher.polygon_geohasher import geohash_to_polygon
# import geopandas as gpd
# import geohash
# locations = [(37.49786897152, 127.02764899362),
#              (37.50732421875, 126.97998046875),
#              (37.50732421875, 127.06787109375),
#              (37.46337890625, 127.02392578125),
#              (37.46337890625, 126.97998046875),
#              (37.46337890625, 127.06787109375),
#              (37.55126953125, 127.02392578125),
#              (37.55126953125, 126.97998046875),
#              (37.55126953125, 127.06787109375)]

# # Create Geo Pandas DataFrame
# ddf = gpd.GeoDataFrame({'location':ex_df['poly'], 'value': np.random.rand(len(ex_df))})
# ddf['geohash'] = ddf['location'].apply(lambda l: geohash.encode(l[0], l[1], 5))
# ddf['geometry'] = ddf['geohash'].apply(geohash_to_polygon)
# ddf.crs = {'init': 'epsg:4326'}


# print('features.properties.geohash <- 요걸로 매핑함')
# display(json.loads(ddf.to_json())['features'][0])
# display(ddf.head())

In [ ]:
YTC = [37.5245456464592, 126.964197674537]

map_test = folium.Map(location=YTC,tiles="openstreetmap",   zoom_start=11)
folium.GeoJson(data=ex_df["polygon"]).add_to(map_test)
map_test

In [ ]:
YTC = [37.5245456464592, 126.964197674537]

geo_path = '02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map_starbucks = folium.Map(location=YTC, zoom_start=11)
map_starbucks.choropleth(geo_data = geo_str,
              data = ex_df['인원수'],
              columns=[ex_df.지역, ex_df['인원수']],
              fill_color='PuRd',
              key_on='feature.id')
map_starbucks